In [501]:
from pytrack.data_mgmt import * 
from pytrack.game_moments import * 
from pytrack.positions_analysis import * 
from pytrack.pitch_creation import * 
from pytrack.build_up import * 
from tracking_config import *

#from moviepy.editor import VideoFileClip, vfx
import glob
from google.cloud import bigquery
from pandas.io import gbq
import pandas_gbq
import plotly_express as px
import matplotlib.pyplot as plt
from forel_clustering_master import *
import os
import sys
import bisect
from bisect import bisect
from sklearn.cluster import KMeans
from shapely.geometry import Point, Polygon, MultiPoint
import moviepy.video.io.ImageSequenceClip
from PIL import Image
import warnings

go = 3

In [502]:
def filter_find_game_id_f7(homeTeam_str, awayTeam_str, competitionCountry, competitionName__, competitionSearchFlag, matchday, seasonId, month_, avoid_under, avoid_women_teams, avoid_women_under):

    if competitionSearchFlag == 'exact':
        base_query = 'SELECT competitionCountry, optaCompName, game_id, stat_matchday, matchDate, homeTeamName, awayTeamName FROM `the-fa-analysis-and-insight.source_external_opta.opta_f7_detailed_table` WHERE (optaCompName = "'+str(competitionName__)+'" AND EXTRACT(YEAR FROM matchDate) = '+str(seasonId)+')'
        
    else:
        base_query = 'SELECT competitionCountry, optaCompName, game_id, stat_matchday, matchDate, homeTeamName, awayTeamName FROM `the-fa-analysis-and-insight.source_external_opta.opta_f7_detailed_table` WHERE (optaCompName LIKE "%'+str(competitionName__)+'%" AND EXTRACT(YEAR FROM matchDate) = '+str(seasonId)+')'

    if month_ != '':
        base_query = base_query[:-1] + ' AND EXTRACT(MONTH FROM matchDate) = '+str(month_)+')'
        
    print(competitionName)
    # Add conditions based on the provided variables
    conditions = []
    
    if homeTeam_str != '':
        conditions.append(f"homeTeamName LIKE '%{homeTeam_str}%'")
    if awayTeam_str != '':
        conditions.append(f"awayTeamName LIKE '%{awayTeam_str}%'")
    if competitionCountry != '':
        conditions.append(f"competitionCountry LIKE '%{competitionCountry}%'")
    if matchday != '':
        conditions.append(f"stat_matchday = {matchday}")
    if avoid_women_teams == 'Yes':
            conditions.append(f"homeTeamName NOT LIKE '%Women%'")
            conditions.append(f"awayTeamName NOT LIKE '%Women%'")
        
    if avoid_women_under == 'Yes':
        for under_team in ['U23','U21','U20','U19','U18','U17','U16','U15']:
            conditions.append(f"homeTeamName NOT LIKE '%Women {under_team}%'")
            conditions.append(f"awayTeamName NOT LIKE '%Women {under_team}%'")
   
    if avoid_under == 'Yes':
        for under_team in ['U23','U21','U20','U19','U18','U17','U16','U15']:
            conditions.append(f"homeTeamName NOT LIKE '%{under_team}%'")
            conditions.append(f"awayTeamName NOT LIKE '%{under_team}%'")
    elif avoid_under == 'Only':
        for under_team in ['U23','U20','U18','U16','U15']:
            conditions.append(f"homeTeamName NOT LIKE '%{under_team}%'")
            conditions.append(f"awayTeamName NOT LIKE '%{under_team}%'")
    
    
    # Combine the base query with the conditions
    if conditions:
        base_query += " AND " + " AND ".join(conditions)
    
    df_fixtures = pandas_gbq.read_gbq(base_query, project_id = 'the-fa-analysis-and-insight')
    
    df_fixtures_r = df_fixtures.rename(columns = {'optaCompName':'competitionName', 'game_id':'matchId', 'stat_matchday':'matchday', 'matchDate':'matchDateTime', 'homeTeamName':'homeTeam', 'awayTeamName':'awayTeam'})
    
    df_fixtures_rs = df_fixtures_r[['competitionCountry', 'competitionName', 'matchId', 'matchday', 'matchDateTime', 'homeTeam', 'awayTeam']].drop_duplicates()
    
    return df_fixtures_rs

In [503]:
if go == 1:
    team_name = 'England U21'
    replaceOrAppend = 'replace'
    filter_benchmark = '21'

elif go == 2:
    team_name = 'England U18'
    replaceOrAppend = 'append'
    filter_benchmark = '18'

elif go == 3:
    team_name = 'England U17'
    replaceOrAppend = 'append'

else:
    pass


if team_name == 'England U21' :
    keep_competitions =  ['UEFA Under 21 Championship', 'UEFA Under 21 Championship Qualifiers', 'Under 21 Internationals']

else:
    keep_competitions =  ['FIFA U17 World Cup','Under 18 Internationals'] 



In [504]:

avoid_under = 'No' #Yes = Only Senior, #Only = Sen, U21, U19, U17, #No = all teams
avoid_women_teams = 'No'
avoid_women_under = 'No'
data_version_label = '1.3.19'

find_game_id_filt = pd.DataFrame()

test_combos_to_compute = []

month_start = 8
month_end = 12
year = 2025

if type(team_name) == str:

    add_team_name = team_name
    
    for mn in list(np.arange(month_start, month_end+1, 1)):
    
        test_combos_to_compute.append(['',team_name,'','','','',year,mn])
        test_combos_to_compute.append([team_name,'','','','','',year,mn])

elif type(team_name) == list:

    add_team_name = team_name[0]
    
    for team_ in team_name:
        for mn in list(np.arange(month_start, month_end+1, 1)):
        
            test_combos_to_compute.append(['',team_,'','','','',year,mn])
            test_combos_to_compute.append([team_,'','','','','',year,mn])

else:
    print('ERROR')

for combo in test_combos_to_compute:
    
        [homeTeam_str,awayTeam_str,competitionCountry,competitionName,competitionSearchFlag,matchday,seasonId, month_] = combo
        
        app_find_game_id_filt = filter_find_game_id_f7(homeTeam_str, awayTeam_str, competitionCountry, competitionName, competitionSearchFlag, matchday, seasonId, month_, avoid_under, avoid_women_teams, avoid_women_under)
            
        # Reset indices of both DataFrames
        find_game_id_filt.reset_index(drop=True, inplace=True)
        app_find_game_id_filt.reset_index(drop=True, inplace=True)
        print('')
        print('Appended '+str(combo))
        # Concatenate DataFrames while ignoring the index
        find_game_id_filt = pd.concat([find_game_id_filt, app_find_game_id_filt], ignore_index=True)
        find_game_id_filt = find_game_id_filt.drop_duplicates()

if keep_competitions != []:
    find_game_id_filt = find_game_id_filt[find_game_id_filt['competitionName'].isin(keep_competitions)]

print(find_game_id_filt.sort_values(by = ['matchDateTime'], ascending = False))


Downloading: |                                                                 |

Appended ['', 'England U17', '', '', '', '', 2025, np.int64(8)]

Downloading: |                                                                 |

Appended ['England U17', '', '', '', '', '', 2025, np.int64(8)]

Downloading: 100%|█████████████████████████████████████████████████████████████|

Appended ['', 'England U17', '', '', '', '', 2025, np.int64(9)]

Downloading: 100%|█████████████████████████████████████████████████████████████|

Appended ['England U17', '', '', '', '', '', 2025, np.int64(9)]

Downloading: |                                                                 |

Appended ['', 'England U17', '', '', '', '', 2025, np.int64(10)]

Downloading: 100%|█████████████████████████████████████████████████████████████|

Appended ['England U17', '', '', '', '', '', 2025, np.int64(10)]

Downloading: 100%|█████████████████████████████████████████████████████████████|

Appended ['', 'England U17', '', 

In [505]:
games_from_opta = find_game_id_filt[['matchDateTime','homeTeam','awayTeam']].drop_duplicates()
list_of_games_from_opta = list(map(lambda d : [str(d[0]),d[1],d[2]], games_from_opta.values.tolist()))

list_of_games_from_opta

[['2025-11-18', 'Austria U17', 'England U17'],
 ['2025-11-10', 'Egypt U17', 'England U17'],
 ['2025-11-15', 'Korea Republic U17', 'England U17'],
 ['2025-11-07', 'England U17', 'Haiti U17'],
 ['2025-11-04', 'England U17', 'Venezuela U17']]

In [506]:
games_string_joined = ', '.join(map(str, list(find_game_id_filt.sort_values(by = ['matchDateTime'])['matchId'].unique())))
games_string_joined

'2556968, 2556970, 2556972, 2608983, 2609005'

In [507]:
opp_strengths_query = """
SELECT *
FROM `the-fa-performance-insights.dev_vanni.senior_team_tier`
"""

def join_t3_t4(Tier):

    if Tier >=3 :
        return 3
    else:
        return Tier
        
opp_strengths = pandas_gbq.read_gbq(opp_strengths_query, project_id = 'the-fa-analysis-and-insight')
opp_strengths_senior= opp_strengths[opp_strengths['TeamType'] == "Men's Senior"]
opp_strengths_senior['Tier'] = opp_strengths_senior.apply(lambda row: join_t3_t4(row.Tier), axis = 1)

display_all(opp_strengths_senior.groupby(by = ['Tier']).count())

Downloading: 100%|█████████████████████████████████████████████████████████████|


,TeamName,senior_team_rank,TeamType
Tier,,,
1,10,10,10
2,20,20,20
3,183,183,183


In [508]:
benchmark_query = """
SELECT DISTINCT *
FROM `the-fa-performance-insights.dev_vanni.benchmark_phases_by_games`
"""

benchmark_games_pre_f = pandas_gbq.read_gbq(benchmark_query, project_id = 'the-fa-performance-insights')

if go == 1:
    benchmark_games = benchmark_games_pre_f[benchmark_games_pre_f['event_team_name'].str.contains('21')]
else:
    benchmark_games = benchmark_games_pre_f[~benchmark_games_pre_f['event_team_name'].str.contains('21')]
    
benchmark_games_list = list(map(lambda d : str(d), benchmark_games.game_id.unique()))
benchmark_games_list_str = ', ' .join(benchmark_games_list)

benchmark_games['team_tier'] = benchmark_games.apply(lambda row: join_t3_t4(row.team_tier), axis = 1)
benchmark_games['against_tier'] = benchmark_games.apply(lambda row: join_t3_t4(row.against_tier), axis = 1)

benchmark_games.groupby(by = ['game_id','team_tier','against_tier']).count()

Downloading: 100%|█████████████████████████████████████████████████████████████|


/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3717996945.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark_games['team_tier'] = benchmark_games.apply(lambda row: join_t3_t4(row.team_tier), axis = 1)
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3717996945.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark_games['against_tier'] = benchmark_games.apply(lambda row: join_t3_t4(row.against_tier), axis = 1)


event_team_name  against_team_name  phase  \
game_id team_tier against_tier                                              
2200777 1.0       3.0                         6                  6      6   
        3.0       1.0                         6                  6      6   
2245009 1.0       3.0                         6                  6      6   
        3.0       1.0                         6                  6      6   
2256819 1.0       2.0                         6                  6      6   
...                                         ...                ...    ...   
2609009 3.0       3.0                        12                 12     12   
2609010 2.0       3.0                         6                  6      6   
        3.0       2.0                         6                  6      6   
2609011 2.0       3.0                         6                  6      6   
        3.0       2.0                         6                  6      6   

                                time_diff  KPI  Value  n_of_matches  \
game_id team_tier against_tier                                        
2200777 1.0       3.0                   6    6      6             6   
        3.0       1.0                   6    6      6             6   
2245009 1.0       3.0                   6    6      6             6   
        3.0       1.0                   6    6      6             6   
2256819 1.0       2.0                   6    6      6             6   
...                                   ...  ...    ...           ...   
2609009 3.0       3.0                  12   12     12            12   
2609010 2.0       3.0                   6    6      6             6   
        3.0       2.0                   6    6      6             6   
2609011 2.0       3.0                   6    6      6             6   
        3.0       2.0                   6    6      6             6   

                                total_create_to_finish  \
game_id team_tier against_tier                           
2200777 1.0       3.0                                0   
        3.0       1.0                                0   
2245009 1.0       3.0                                0   
        3.0       1.0                                0   
2256819 1.0       2.0                                0   
...                                                ...   
2609009 3.0       3.0                                0   
2609010 2.0       3.0                                0   
        3.0       2.0                                0   
2609011 2.0       3.0                                0   
        3.0       2.0                                0   

                                successful_create_to_finish  
game_id team_tier against_tier                               
2200777 1.0       3.0                                     0  
        3.0       1.0                                     0  
2245009 1.0       3.0                                     0  
        3.0       1.0                                     0  
2256819 1.0       2.0                                     0  
...                                                     ...  
2609009 3.0       3.0                                     0  
2609010 2.0       3.0                                     0  
        3.0       2.0                                     0  
2609011 2.0       3.0                                     0  
        3.0       2.0                                     0  

[338 rows x 9 columns]

In [509]:
benchmark_games_list_str

'2366349, 2200777, 2245009, 2256819, 2256820, 2258557, 2258809, 2258810, 2283809, 2283810, 2286817, 2286818, 2286819, 2332091, 2332217, 2332280, 2359950, 2360022, 2360129, 2365745, 2365746, 2365747, 2409197, 2409297, 2409353, 2411546, 2411547, 2411548, 2431914, 2438006, 2438169, 2441364, 2441418, 2485839, 2485886, 2486058, 2488539, 2488588, 2490038, 2490778, 2514201, 2514202, 2514864, 2555022, 2602848, 2602931, 2602932, 2606776, 2606777, 2605712, 2605713, 2607979, 2607980, 2409809, 2409821, 2409834, 2556999, 2557000, 2557003, 2409822, 2409833, 2409849, 2409856, 2556950, 2556953, 2556955, 2608971, 2608990, 2609007, 2409810, 2556987, 2556988, 2556991, 2409845, 2409811, 2409823, 2409835, 2409846, 2409853, 2409824, 2409836, 2557005, 2557006, 2557009, 2608977, 2409812, 2409850, 2409855, 2556998, 2557001, 2608979, 2609006, 2409851, 2409858, 2409859, 2557010, 2557012, 2557014, 2608976, 2409813, 2409825, 2409837, 2409854, 2556986, 2556990, 2608978, 2608991, 2608993, 2609008, 2409826, 2409838, 

In [510]:
benchmark_games[['event_team_name','against_team_name']].drop_duplicates()

,event_team_name,against_team_name
1706,Japan U22,England U21
1712,Japan U22,None
1831,England U18,Wales U18
1837,England U18,Norway U18
1840,England U18,Russia U18
...,...,...
4723,Hungary U18,Bosnia and Herzegovina U18
4726,Hungary U18,Slovakia U18
4735,Hungary U18,None
4737,Slovakia U18,Hungary U18


In [511]:
eng_benchmark_games = benchmark_games[ (benchmark_games['event_team_name'].str.contains('England'))]
display_all(eng_benchmark_games.sort_values(by = ['event_team_name','against_team_name','phase']).head(30))
display_all(benchmark_games.sort_values(by = ['event_team_name','against_team_name','phase']).head(30))

print(eng_benchmark_games['game_id'].unique())
print(benchmark_games['game_id'].unique())

,event_team_name,game_id,against_team_name,team_tier,against_tier,phase,time_diff,KPI,Value,n_of_matches,total_create_to_finish,successful_create_to_finish
3079,England U17,2609005,Austria U17,1.0,2.0,% IN HIGH BLOCK,257.152,% IN HIGH BLOCK,10.72,9,NaN,NaN
3081,England U17,2609005,Austria U17,1.0,2.0,% IN LOW BLOCK,493.835,% IN LOW BLOCK,20.58,9,NaN,NaN
3080,England U17,2609005,Austria U17,1.0,2.0,% IN MID BLOCK,317.409,% IN MID BLOCK,13.23,9,NaN,NaN
3052,England U17,2609005,Austria U17,1.0,2.0,BUILD,274.635,% IN BUILD PHASE,11.45,9,NaN,NaN
3053,England U17,2609005,Austria U17,1.0,2.0,CREATE,497.650,% IN CREATE PHASE,20.74,9,NaN,NaN
3054,England U17,2609005,Austria U17,1.0,2.0,FINISH,558.558,% IN FINISH PHASE,23.28,9,NaN,NaN
3061,England U17,2409837,Brazil U17,1.0,1.0,% IN HIGH BLOCK,159.094,% IN HIGH BLOCK,5.16,9,NaN,NaN
3063,England U17,2409837,Brazil U17,1.0,1.0,% IN LOW BLOCK,367.998,% IN LOW BLOCK,11.94,9,NaN,NaN
3062,England U17,2409837,Brazil U17,1.0,1.0,% IN MID BLOCK,485.733,% IN MID BLOCK,15.76,9,NaN,NaN
3034,England U17,2409837,Brazil U17,1.0,1.0,BUILD,310.346,% IN BUILD PHASE,10.07,9,NaN,NaN


,event_team_name,game_id,against_team_name,team_tier,against_tier,phase,time_diff,KPI,Value,n_of_matches,total_create_to_finish,successful_create_to_finish
3145,Argentina U17,2556962,Belgium U17,1.0,1.0,% IN HIGH BLOCK,281.714,% IN HIGH BLOCK,12.07,11,NaN,NaN
3147,Argentina U17,2556962,Belgium U17,1.0,1.0,% IN LOW BLOCK,279.014,% IN LOW BLOCK,11.95,11,NaN,NaN
3146,Argentina U17,2556962,Belgium U17,1.0,1.0,% IN MID BLOCK,580.463,% IN MID BLOCK,24.87,11,NaN,NaN
3112,Argentina U17,2556962,Belgium U17,1.0,1.0,BUILD,219.253,% IN BUILD PHASE,9.39,11,NaN,NaN
3113,Argentina U17,2556962,Belgium U17,1.0,1.0,CREATE,532.342,% IN CREATE PHASE,22.80,11,NaN,NaN
3114,Argentina U17,2556962,Belgium U17,1.0,1.0,FINISH,441.613,% IN FINISH PHASE,18.92,11,NaN,NaN
3136,Argentina U17,2409854,Brazil U17,1.0,1.0,% IN HIGH BLOCK,246.495,% IN HIGH BLOCK,8.96,11,NaN,NaN
3138,Argentina U17,2409854,Brazil U17,1.0,1.0,% IN LOW BLOCK,275.372,% IN LOW BLOCK,10.00,11,NaN,NaN
3137,Argentina U17,2409854,Brazil U17,1.0,1.0,% IN MID BLOCK,699.275,% IN MID BLOCK,25.41,11,NaN,NaN
3103,Argentina U17,2409854,Brazil U17,1.0,1.0,BUILD,271.414,% IN BUILD PHASE,9.86,11,NaN,NaN


<IntegerArray>
[2200777, 2245009, 2256819, 2256820, 2258557, 2258809, 2258810, 2283809,
 2283810, 2286817, 2286818, 2286819, 2332091, 2332217, 2332280, 2359950,
 2360022, 2360129, 2365745, 2365746, 2365747, 2409197, 2409297, 2409353,
 2411546, 2411547, 2411548, 2431914, 2438006, 2438169, 2441364, 2441418,
 2485839, 2485886, 2486058, 2488539, 2488588, 2490038, 2490778, 2514201,
 2514202, 2514864, 2555022, 2602848, 2602931, 2602932, 2606776, 2606777,
 2409814, 2409826, 2409837, 2409850, 2556968, 2556970, 2556972, 2608983,
 2609005]
Length: 57, dtype: Int64
<IntegerArray>
[2366349, 2200777, 2245009, 2256819, 2256820, 2258557, 2258809, 2258810,
 2283809, 2283810,
 ...
 2556983, 2556985, 2556981, 2556989, 2556993, 2557002, 2557007, 2557011,
 2607367, 2608162]
Length: 210, dtype: Int64


In [512]:
def find_create_to_finish_metrics_by_tier(data, tier):

    new_df_f_out_teamed = data[(data['team_tier'] == tier)]
    new_df_f_out_opponents = data[(data['against_tier'] == tier)]

    n_of_matches = len(new_df_f_out_teamed[['event_team_name','game_id']].drop_duplicates())
    create_to_finish_successful = new_df_f_out_teamed[(new_df_f_out_teamed['CREATE_FINISH'] != '')].groupby(by = ['team_tier'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'successful_create_to_finish'})
    create_to_finish_total = new_df_f_out_teamed.groupby(by = ['team_tier'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'total_create_to_finish'})
    create_to_finish_metrics = pd.merge(create_to_finish_total, create_to_finish_successful, on = ['team_tier', 'game_id'])
    
    opp_n_of_matches = len(new_df_f_out_teamed[['against_team_name','game_id']].drop_duplicates())
    opp_create_to_finish_successful = new_df_f_out_opponents[(new_df_f_out_opponents['CREATE_FINISH'] != '')].groupby(by = ['against_tier'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'opp_successful_create_to_finish'})
    opp_create_to_finish_total = new_df_f_out_opponents.groupby(by = ['against_tier'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'opp_total_create_to_finish'})
    opp_create_to_finish_metrics = pd.merge(opp_create_to_finish_total, opp_create_to_finish_successful, on = ['against_tier'])
    
    teamed_create_to_finish_metrics = create_to_finish_metrics[(create_to_finish_metrics['event_team_name'] == tier)]
    opp_create_to_finish_metrics = opp_create_to_finish_metrics[(opp_create_to_finish_metrics['against_team_name'] == tier)]

    teamed_create_to_finish_metrics['Value'] = (teamed_create_to_finish_metrics['successful_create_to_finish']  / teamed_create_to_finish_metrics['total_create_to_finish']  * 100).round(2)
    opp_create_to_finish_metrics['Value'] = (opp_create_to_finish_metrics['opp_successful_create_to_finish']  / opp_create_to_finish_metrics['opp_total_create_to_finish']  * 100).round(2)
    
    teamed_create_to_finish_metrics['KPI'] = add_prefix+'Create to Finish Success Rate (%)'
    teamed_create_to_finish_metrics['n_of_matches'] = n_of_matches

    opp_create_to_finish_metrics['KPI'] = add_prefix+'Opponent Create to Finish Success Rate (%)'
    opp_create_to_finish_metrics['n_of_matches'] = opp_n_of_matches
    opp_create_to_finish_metrics = opp_create_to_finish_metrics.rename(columns = {'against_tier':'team_tier'})

    out_create_to_finish_metrics = pd.concat([teamed_create_to_finish_metrics,opp_create_to_finish_metrics[teamed_create_to_finish_metrics.columns]], ignore_index = True)
    
    return out_create_to_finish_metrics

def find_time_metrics_by_tier(data, tier):

    new_df_f_out_teamed = data[(data['team_tier'] == tier)]
    new_df_f_out_opponents = data[(data['against_tier'] == tier)]

    n_of_matches = len(new_df_f_out_teamed[['event_team_name','game_id']].drop_duplicates())
    in_possession_time_spent_per_team_phase = new_df_f_out_teamed[new_df_f_out_teamed['phase'].isin(['BUILD','CREATE','FINISH'])].groupby(by = ['team_tier','against_tier', 'phase'])['time_diff'].sum().reset_index()
    out_of_possession_time_spent_per_team_phase = new_df_f_out_teamed[new_df_f_out_teamed['phase'].isin(['% IN LOW BLOCK','% IN MID BLOCK','% IN HIGH BLOCK'])].groupby(by = ['team_tier','against_tier', 'phase'])['time_diff'].sum().reset_index()
    
    out_of_possession_time_spent_per_team_phase = out_of_possession_time_spent_per_team_phase[in_possession_time_spent_per_team_phase.columns]
    
    time_spent_metrics = pd.concat([in_possession_time_spent_per_team_phase, out_of_possession_time_spent_per_team_phase])
    
    time_spent_metrics['KPI'] = time_spent_metrics['phase'].replace({
        'BUILD': '% IN BUILD PHASE',
        'CREATE': '% IN CREATE PHASE',
        'FINISH': '% IN FINISH PHASE'
    })
    
    time_spent_metrics['Value'] = (time_spent_metrics['time_diff'] / time_spent_metrics.groupby(by = ['against_tier','team_tier'])['time_diff'].transform('sum') * 100).round(2)
    
    teamed_time_metrics = time_spent_metrics[(time_spent_metrics['team_tier'] == tier)]
    teamed_time_metrics['n_of_matches'] = n_of_matches
    
    return teamed_time_metrics

tier_team_metrics = pd.DataFrame()
eng_tier_team_metrics = pd.DataFrame()

for tier_ in benchmark_games['team_tier'].unique():

    print('benchmark_games tier:')
    print(tier_)
    
    add_tier_team_metrics = find_time_metrics_by_tier(benchmark_games, tier_)
    print('Length of add_tier_team_metrics: '+str(len(add_tier_team_metrics)))
    print(len(add_tier_team_metrics))
    
    tier_team_metrics = pd.concat([tier_team_metrics, add_tier_team_metrics], ignore_index = True)

for tier_ in eng_benchmark_games['team_tier'].unique():

    print('eng_benchmark_games tier:')
    print(tier_)
    
    add_eng_tier_team_metrics = find_time_metrics_by_tier(eng_benchmark_games, tier_)
    print('Length of add_eng_tier_team_metrics: '+str(len(add_eng_tier_team_metrics)))
    
    eng_tier_team_metrics = pd.concat([eng_tier_team_metrics, add_eng_tier_team_metrics], ignore_index = True)

display_all(tier_team_metrics.sort_values(by = ['team_tier','phase','against_tier']).head(30))

benchmark_games tier:
2.0
Length of add_tier_team_metrics: 18
18
benchmark_games tier:
nan
Length of add_tier_team_metrics: 0
0
benchmark_games tier:
1.0
Length of add_tier_team_metrics: 18
18
benchmark_games tier:
3.0
Length of add_tier_team_metrics: 18
18
eng_benchmark_games tier:
1.0
Length of add_eng_tier_team_metrics: 18
eng_benchmark_games tier:
nan
Length of add_eng_tier_team_metrics: 0


,team_tier,against_tier,phase,time_diff,KPI,Value,n_of_matches
27,1.0,1.0,% IN HIGH BLOCK,24289.181,% IN HIGH BLOCK,14.18,153
30,1.0,2.0,% IN HIGH BLOCK,14074.767,% IN HIGH BLOCK,12.98,153
33,1.0,3.0,% IN HIGH BLOCK,15929.608,% IN HIGH BLOCK,10.66,153
28,1.0,1.0,% IN LOW BLOCK,23743.715,% IN LOW BLOCK,13.86,153
31,1.0,2.0,% IN LOW BLOCK,10781.798,% IN LOW BLOCK,9.94,153
34,1.0,3.0,% IN LOW BLOCK,13281.592,% IN LOW BLOCK,8.89,153
29,1.0,1.0,% IN MID BLOCK,37636.342,% IN MID BLOCK,21.97,153
32,1.0,2.0,% IN MID BLOCK,16528.468,% IN MID BLOCK,15.24,153
35,1.0,3.0,% IN MID BLOCK,22712.630,% IN MID BLOCK,15.20,153
18,1.0,1.0,BUILD,24289.181,% IN BUILD PHASE,14.18,153


In [513]:
display_all(eng_tier_team_metrics.sort_values(by = ['team_tier','phase','against_tier']).head(30))

,team_tier,against_tier,phase,time_diff,KPI,Value,n_of_matches
9,1.0,1.0,% IN HIGH BLOCK,9837.422,% IN HIGH BLOCK,16.48,57
12,1.0,2.0,% IN HIGH BLOCK,8323.923,% IN HIGH BLOCK,13.47,57
15,1.0,3.0,% IN HIGH BLOCK,6237.249,% IN HIGH BLOCK,13.17,57
10,1.0,1.0,% IN LOW BLOCK,7074.412,% IN LOW BLOCK,11.85,57
13,1.0,2.0,% IN LOW BLOCK,5763.683,% IN LOW BLOCK,9.32,57
16,1.0,3.0,% IN LOW BLOCK,3563.063,% IN LOW BLOCK,7.52,57
11,1.0,1.0,% IN MID BLOCK,10864.506,% IN MID BLOCK,18.20,57
14,1.0,2.0,% IN MID BLOCK,8251.695,% IN MID BLOCK,13.35,57
17,1.0,3.0,% IN MID BLOCK,7684.932,% IN MID BLOCK,16.23,57
0,1.0,1.0,BUILD,8388.299,% IN BUILD PHASE,14.05,57


In [514]:
tier_team_metrics['event_team_name'] = add_team_name
tier_team_metrics['benchmark_type'] = 'Tier Comparison'

eng_tier_team_metrics['event_team_name'] = add_team_name
eng_tier_team_metrics['benchmark_type'] = 'England Benchmark'

phases_by_tier_metrics = pd.concat([eng_tier_team_metrics, tier_team_metrics], ignore_index = True)

display_all(phases_by_tier_metrics.head())

,team_tier,against_tier,phase,time_diff,KPI,Value,n_of_matches,event_team_name,benchmark_type
0,1.0,1.0,BUILD,8388.299,% IN BUILD PHASE,14.05,57,England U17,England Benchmark
1,1.0,1.0,CREATE,14363.974,% IN CREATE PHASE,24.06,57,England U17,England Benchmark
2,1.0,1.0,FINISH,9178.738,% IN FINISH PHASE,15.37,57,England U17,England Benchmark
3,1.0,2.0,BUILD,10490.139,% IN BUILD PHASE,16.97,57,England U17,England Benchmark
4,1.0,2.0,CREATE,17114.765,% IN CREATE PHASE,27.69,57,England U17,England Benchmark


In [515]:
team_data = pd.read_csv('season_export_murph_dropped.csv')

team_data['string_date'] = team_data.apply(lambda row: row['date - opposition'].split(' -')[0], axis = 1)


In [516]:
team_data

,Competition,date,date - opposition,Empty,Macro Filter average games,Macro Filter games,Opposition,opposition_scorers,result,scoreline_colour,...,game_id,goals,goals_conceded,possession,possession_conceded,shots,shots_conceded,xg,xg_conceded,string_date
0,Under 17 Internationals,08/08/2022,2022-08-08 - Faroe Islands U17,,No,No,Faroe Islands U17,NaN,3 - 0,green,...,2324245,3.0,0.0,0.777377,0.222623,26,2,2.114039,0.175184,2022-08-08
1,UEFA Under 17 Championship Qualifiers,19/10/2021,2021-10-19 - Belarus U17,,No,No,Belarus U17,NaN,1 - 0,green,...,2239954,1.0,0.0,0.813449,0.186551,11,5,0.634132,0.194528,2021-10-19
2,Under 20 Internationals,21/03/2025,2025-03-21 - Portugal U20,,No,No,Portugal U20,Diogo Monteiro (1),1 - 1,amber,...,2514276,1.0,1.0,0.539150,0.460850,20,11,3.696557,1.617830,2025-03-21
3,UEFA Under 17 Championship,15/05/2016,2016-05-15 - Spain U17,,No,No,Spain U17,Fran García (1),0 - 1,red,...,852589,0.0,1.0,0.548422,0.451578,10,8,1.743230,0.943152,2016-05-15
4,Under 16 Internationals,15/12/2024,2024-12-15 - Netherlands U16,,No,No,Netherlands U16,Jeshurun Simeon (1),2 - 1,green,...,2491005,2.0,1.0,0.547192,0.452808,13,18,1.784926,2.196878,2024-12-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,Under 18 Internationals,11/10/2024,2024-10-11 - Sweden U18,,Yes,Yes,Sweden U18,Andreas Redkin (2),NaN,NaN,...,2488539,NaN,2.0,0.729979,0.270021,19,11,1.323127,1.285133,2024-10-11
663,Under 16 Internationals,10/04/2023,2023-04-10 - Japan U16,,No,No,Japan U16,NaN,0 - 0,amber,...,2361322,0.0,0.0,0.556688,0.443312,13,10,1.418668,0.929778,2023-04-10
664,UEFA Under 19 Championship,01/07/2022,2022-07-01 - Israel U19,,No,No,Israel U19,Oscar Gloukh (1),3 - 1,green,...,2304880,3.0,1.0,0.440613,0.559387,18,19,3.081790,1.900481,2022-07-01
665,Under 17 Internationals,12/10/2019,2019-10-12 - Germany U17,,No,No,Germany U17,"Lasse Günther (1), Luca Netz (1)",1 - 2,red,...,2027825,1.0,2.0,0.537475,0.462525,6,9,0.598600,1.157530,2019-10-12


In [517]:
filt_team_data = pd.DataFrame()

for game_ in list_of_games_from_opta:
    data_ = game_[0]
    eng_team = list(filter(lambda row: 'England' in row, game_[1:]))[0]
    opp_team = list(filter(lambda row: 'England' not in row, game_[1:]))[0]
    
    find_game = team_data[(team_data['date - opposition'].str.contains(data_)) & (team_data['Team'] == eng_team) & (team_data['Opposition'] == opp_team)]
    
    if len(find_game) > 0:
        print('Found '+str(len(find_game))+' game/s')
        filt_team_data = pd.concat([filt_team_data, find_game], ignore_index = True)

if type(team_name) == str:
    filt_team_data['Team'] = team_name
elif type(team_name) == list:
    filt_team_data['Team'] = team_name[0]

aggr_stats_avg = filt_team_data.groupby(by = ['Team'])[['goals','goals_conceded','xg','xg_conceded']].mean().reset_index().rename(columns = {'goals':'avg_goals','xg_conceded':'avg_xg_conceded','xg':'avg_xg','goals_conceded':'avg_goals_conceded'})
aggr_stats_sum = filt_team_data.groupby(by = ['Team'])[['goals','goals_conceded','xg','xg_conceded']].sum().reset_index()

aggr_stats = pd.merge(aggr_stats_avg, aggr_stats_sum, on = ['Team'], how = 'left')

display(aggr_stats)

Found 1 game/s
Found 1 game/s
Found 1 game/s
Found 1 game/s
Found 1 game/s


,Team,avg_goals,avg_goals_conceded,avg_xg,avg_xg_conceded,goals,goals_conceded,xg,xg_conceded
0,England U17,2.6,1.6,2.427684,1.899687,13.0,8.0,12.138418,9.498435


In [518]:

if type(team_name) == str:
    aggr_stats['Team'] = team_name
elif type(team_name) == list:
    aggr_stats['Team'] = team_name[0]

display(aggr_stats)

,Team,avg_goals,avg_goals_conceded,avg_xg,avg_xg_conceded,goals,goals_conceded,xg,xg_conceded
0,England U17,2.6,1.6,2.427684,1.899687,13.0,8.0,12.138418,9.498435


In [519]:
team_name

'England U17'

In [520]:
if type(team_name) == str:

    sql_aggr_phases_metrics = "SELECT DISTINCT * FROM `the-fa-performance-insights.dev_vanni.phases_metrics_aggregate` WHERE event_team_name = '"+str(team_name)+"' and value_type = 'All Matches'"
    
    pre_aggr_phases_metrics = pandas_gbq.read_gbq(sql_aggr_phases_metrics, project_id = 'the-fa-analysis-and-insight')

    aggr_phases_metrics = pre_aggr_phases_metrics[['event_team_name','KPI','time_diff','Value','n_of_matches']]

    print('done!')

elif (type(team_name) == list and team_name[0] == 'England U18'):

    sql_aggr_phases_metrics = "SELECT DISTINCT * FROM `the-fa-performance-insights.dev_vanni.phases_metrics_aggregate` WHERE event_team_name IN ('England U18','England U17') and value_type = 'All Matches'"

    pre_aggr_phases_metrics = pandas_gbq.read_gbq(sql_aggr_phases_metrics, project_id = 'the-fa-analysis-and-insight')

    if type(team_name) == str:
        pre_aggr_phases_metrics['event_team_name'] = team_name
    elif type(team_name) == list:
        pre_aggr_phases_metrics['event_team_name'] = team_name[0]
    
    block_aggr_phases_metrics = pre_aggr_phases_metrics[~pre_aggr_phases_metrics['KPI'].str.contains('Success')][['event_team_name','KPI','time_diff','n_of_matches']].groupby(by = ['event_team_name','KPI'])['time_diff'].sum().reset_index()

    success_aggr_phases_metrics = pre_aggr_phases_metrics[pre_aggr_phases_metrics['KPI'].str.contains('Success')][['event_team_name','KPI','time_diff','Value','n_of_matches']].groupby(by = ['event_team_name','KPI'])[['time_diff','Value']].mean().reset_index()

    block_aggr_phases_metrics['Value'] = 100*(block_aggr_phases_metrics['time_diff'] / block_aggr_phases_metrics.groupby('event_team_name')['time_diff'].transform('sum'))

    aggr_phases_metrics = pd.concat([block_aggr_phases_metrics,success_aggr_phases_metrics], ignore_index = True)
    
display_all(aggr_phases_metrics)

Downloading: 100%|█████████████████████████████████████████████████████████████|
done!


,event_team_name,KPI,time_diff,Value,n_of_matches
0,England U17,% IN BUILD PHASE,1983.164,15.663419,5
1,England U17,% IN CREATE PHASE,3218.633,25.421396,5
2,England U17,% IN FINISH PHASE,2124.512,16.779814,5
3,England U17,% IN HIGH BLOCK,1695.088,13.388138,5
4,England U17,% IN MID BLOCK,2018.865,15.945393,5
5,England U17,% IN LOW BLOCK,1620.856,12.801839,5
6,England U17,Create to Finish Success Rate (%),0.000,25.190000,5
7,England U17,Opponent Create to Finish Success Rate (%),0.000,21.270000,5


In [521]:
display_all(tier_team_metrics.head())

,team_tier,against_tier,phase,time_diff,KPI,Value,n_of_matches,event_team_name,benchmark_type
0,2.0,1.0,BUILD,14479.807,% IN BUILD PHASE,12.98,84,England U17,Tier Comparison
1,2.0,1.0,CREATE,17043.393,% IN CREATE PHASE,15.28,84,England U17,Tier Comparison
2,2.0,1.0,FINISH,11065.801,% IN FINISH PHASE,9.92,84,England U17,Tier Comparison
3,2.0,2.0,BUILD,3200.544,% IN BUILD PHASE,11.44,84,England U17,Tier Comparison
4,2.0,2.0,CREATE,6104.383,% IN CREATE PHASE,21.81,84,England U17,Tier Comparison


In [522]:
sql_wyscout_game_ids = 'SELECT * FROM `the-fa-performance-insights.staging_mapping.opta_wyscout_matches` WHERE opta_match_id IN ('+str(games_string_joined)+')'
wyscout_games = pandas_gbq.read_gbq(sql_wyscout_game_ids, project_id = 'the-fa-analysis-and-insight')

wyscout_games

Downloading: 100%|█████████████████████████████████████████████████████████████|


,opta_match_id,wyscout_match_id,internal_match_id
0,2556970,5784630,57846300
1,2608983,5787246,57872460
2,2556968,5784606,57846060
3,2556972,5784659,57846590
4,2609005,5787709,57877090


In [523]:
sql_benchmark_wyscout_game_ids = 'SELECT * FROM `the-fa-performance-insights.staging_mapping.opta_wyscout_matches` WHERE opta_match_id IN ('+str(benchmark_games_list_str)+')'
benchmark_wyscout_games = pandas_gbq.read_gbq(sql_benchmark_wyscout_game_ids, project_id = 'the-fa-analysis-and-insight')
list_of_benchmark_wyscout_games_string_joined = list(map(lambda a : str(a), benchmark_wyscout_games['internal_match_id'].unique()))
benchmark_wyscout_games_string_joined = ', '.join(list_of_benchmark_wyscout_games_string_joined)


Downloading: 100%|█████████████████████████████████████████████████████████████|


In [524]:
sql_benchmark_events = 'SELECT game_id, Player, Team, Opposition,xG,Open_Play_xG, Shots, Passes, Corners, Big_Chances ,Opta_Passes_in_Own_Half,Possessions_Won_in_Opp_Half,Possessions_Won_in_Final_3rd,Possessions_Won_in_Mid_3rd FROM `the-fa-analysis-and-insight.OPTA_TACT.opta_indi_metrics_app_data` WHERE game_id IN ('+str(benchmark_games_list_str)+')'
benchmark_metrics = pandas_gbq.read_gbq(sql_benchmark_events, project_id = 'the-fa-performance-insights')

benchmark_metrics_games = list(map(lambda a : int(a), benchmark_metrics.game_id.unique()))

display_all(benchmark_metrics.head())

Downloading: 100%|█████████████████████████████████████████████████████████████|


,game_id,Player,Team,Opposition,xG,Open_Play_xG,Shots,Passes,Corners,Big_Chances,Opta_Passes_in_Own_Half,Possessions_Won_in_Opp_Half,Possessions_Won_in_Final_3rd,Possessions_Won_in_Mid_3rd
0,2602932,Richard Chukwu,Canada U18,England U18,0.000000,0.000000,0,38,0,0,30,0,0,0
1,2441364,Calum Moreland,Northern Ireland U18,England U18,0.000000,0.000000,0,10,0,0,9,0,0,0
2,2283810,Oliver Ross,Denmark U18,England U18,0.516263,0.516263,3,9,0,1,4,2,1,1
3,2283810,Tobias Storm,Denmark U18,England U18,0.000000,0.000000,0,1,0,0,1,0,0,0
4,2602931,Asilbek Aliev,Uzbekistan U18,England U18,1.204211,1.142539,4,14,0,2,6,1,1,3


In [525]:
benchmark_metrics[benchmark_metrics['game_id'] == 2602932]

,game_id,Player,Team,Opposition,xG,Open_Play_xG,Shots,Passes,Corners,Big_Chances,Opta_Passes_in_Own_Half,Possessions_Won_in_Opp_Half,Possessions_Won_in_Final_3rd,Possessions_Won_in_Mid_3rd
0,2602932,Richard Chukwu,Canada U18,England U18,0.000000,0.000000,0,38,0,0,30,0,0,0
24,2602932,Aidan Evans,Canada U18,England U18,0.000000,0.000000,0,4,0,0,1,0,0,0
26,2602932,Josh-Duc Nteziryayo,Canada U18,England U18,0.000000,0.000000,0,39,0,0,37,0,0,0
64,2602932,Johnny Selemani,Canada U18,England U18,0.000000,0.000000,0,2,0,0,1,0,0,0
68,2602932,Sahil Deo,Canada U18,England U18,0.052950,0.052950,1,24,1,0,14,3,0,4
122,2602932,Van Parker,Canada U18,England U18,0.088249,0.088249,1,9,0,0,2,1,1,0
127,2602932,Shola Jimoh,Canada U18,England U18,0.223328,0.115627,3,15,0,0,9,4,0,6
152,2602932,Yuma Tsuji,Canada U18,England U18,0.072474,0.072474,1,3,0,0,1,0,0,0
154,2602932,Antoine Nehme,Canada U18,England U18,0.000000,0.000000,0,8,0,0,2,0,0,0
157,2602932,Will Daniels,Canada U18,England U18,0.000000,0.000000,0,5,3,0,1,0,0,0


In [526]:
benchmark_metrics_games[:10]

[2602932,
 2441364,
 2283810,
 2602931,
 2607367,
 2488588,
 2608162,
 2366349,
 2332217,
 2409840]

In [527]:
list_of_wyscout_games_string_joined = list(map(lambda a : str(a), wyscout_games['internal_match_id'].unique()))
wyscout_games_string_joined = ', '.join(list_of_wyscout_games_string_joined)

wyscout_games_string_joined

'57846300, 57872460, 57846060, 57846590, 57877090'

In [528]:
def find_opponent_strength(team_, opponent_):
        
        team_general = team_.replace(' U15', '').replace(' U16', '').replace(' U17', '').replace(' U18', '').replace(' U19', '').replace(' U20', '').replace(' U21', '').replace(' U22', '').replace(' U23', '').replace('Czech Republic','Czechia').replace('USA','United States').replace('IR Iran','Iran').replace('Bosnia and Herzegovina','Bosnia-Herzegovina').replace('DPR Korea','Korea DPR').replace('United Arab Emirates','UAE')
        opponent_general = opponent_.replace(' U15', '').replace(' U16', '').replace(' U17', '').replace(' U18', '').replace(' U19', '').replace(' U20', '').replace(' U21', '').replace(' U22', '').replace(' U23', '').replace('Czech Republic','Czechia').replace('USA','United States').replace('IR Iran','Iran').replace('Bosnia and Herzegovina','Bosnia-Herzegovina').replace('DPR Korea','Korea DPR').replace('United Arab Emirates','UAE')

        team_strengths_filt = opp_strengths_senior[opp_strengths_senior['TeamName'].str.contains(team_general)]
        opp_strengths_filt = opp_strengths_senior[opp_strengths_senior['TeamName'].str.contains(opponent_general)]

        if not team_strengths_filt.empty:
            team_out = team_strengths_filt['Tier'].iloc[0]
        else:
            team_out = None
            
        if not opp_strengths_filt.empty:
            opp_out = opp_strengths_filt['Tier'].iloc[0]
        else:
            opp_out = None

        return [team_out, opp_out]




In [529]:
merged_metrics = benchmark_metrics[benchmark_metrics['game_id'].isin(find_game_id_filt['matchId'].unique())]

merged_metrics[['team_tier', 'against_tier']] = ( merged_metrics.apply(lambda row: find_opponent_strength(row.Team, row.Opposition), axis=1) .apply(pd.Series) )
merged_metrics

display_all(merged_metrics.head())

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3203903931.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_metrics[['team_tier', 'against_tier']] = ( merged_metrics.apply(lambda row: find_opponent_strength(row.Team, row.Opposition), axis=1) .apply(pd.Series) )
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3203903931.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_metrics[['team_tier', 'against_tier']] = ( merged_metrics.apply(lambda row: find_opponent_

,game_id,Player,Team,Opposition,xG,Open_Play_xG,Shots,Passes,Corners,Big_Chances,Opta_Passes_in_Own_Half,Possessions_Won_in_Opp_Half,Possessions_Won_in_Final_3rd,Possessions_Won_in_Mid_3rd,team_tier,against_tier
494,2609005,Kenny Nzogang,Austria U17,England U17,0.000000,0.000000,0,1,0,0,1,0,0,1,2,1
496,2609005,Jakob Pokorny,Austria U17,England U17,0.085657,0.048520,2,39,0,0,34,1,1,0,2,1
501,2609005,Vasilije Markovic,Austria U17,England U17,0.139317,0.139317,2,24,0,0,9,3,2,5,2,1
511,2609005,Luca Weinhandl,Austria U17,England U17,0.257424,0.257424,3,25,0,0,18,0,0,0,2,1
524,2609005,Rafael Feldinger,Austria U17,England U17,0.061668,0.000000,1,36,0,0,12,2,0,4,2,1


In [530]:
stringed_each_single_games_list = ",".join([f"'{g}'" for g in list(find_game_id_filt.sort_values(by = ['matchDateTime'])['matchId'].unique())])
string_benchmark_metrics_games = ",".join([f"'{g}'" for g in benchmark_metrics_games])

string_benchmark_metrics_games

"'2602932','2441364','2283810','2602931','2607367','2488588','2608162','2366349','2332217','2409840','2409297','2286817','2256820','2365747','2488539','2411547','2409828','2608982','2409827','2556966','2409857','2409816','2556964','2409848','2409859','2556962','2409854','2608988','2557012','2608973','2608992','2609005','2608994','2557015','2557011','2556967','2608984','2556965','2360022','2332280','2411546','2258809','2556945','2556948','2556947','2608993','2608991','2556986','2556990','2409813','2556988','2608978','2609008','2409837','2409845','2409825','2608990','2608975','2609009','2556992','2409842','2556995','2556997','2409830','2409817','2608989','2557005','2409811','2557006','2557009','2608977','2409824','2409836','2557007','2557004','2556985','2556983','2608981','2556980','2556958','2556961','2556957','2556956','2556959','2608979','2359950','2286819','2608980','2556993','2556996','2514864','2438006','2556979','2556976','2556975','2556981','2556982','2608972','2609010','2409810'

In [531]:
# ensure strings 
def teamed_control_index(team_, home_, away_, value):

    if team_ == home_:
        return float(value)
    elif team_ == away_:
        return -float(value)
    else:
        return 1000.0
        

sql_momentum_query = ( "SELECT game_id, competition_name, seq, gameClock, seq_period, seq_time_length, game_description, team_name, smoothed_spread_of_ending_value_shot, " " SPLIT(game_description, ' - ')[OFFSET(0)] AS game_home_team_name, " " SPLIT(game_description, ' - ')[OFFSET(1)] AS game_away_team_name, " " CASE " " WHEN team_name = SPLIT(game_description, ' - ')[OFFSET(0)] " " THEN SPLIT(game_description, ' - ')[OFFSET(1)] " " ELSE SPLIT(game_description, ' - ')[OFFSET(0)] " " END AS opponent_team_name, " "FROM `the-fa-analysis-and-insight.dev.momentum_dataset` " "WHERE game_id IN (" + stringed_each_single_games_list + ',' + string_benchmark_metrics_games +")" )
pre_momentum_metrics = pandas_gbq.read_gbq(sql_momentum_query, project_id = 'the-fa-analysis-and-insight')
momentum_metrics = pre_momentum_metrics[pre_momentum_metrics['competition_name'].isin(keep_competitions)]

momentum_metrics[['team_tier', 'against_tier']] = ( momentum_metrics.apply(lambda row: find_opponent_strength(row.team_name, row.opponent_team_name), axis=1) .apply(pd.Series) )
momentum_metrics['teamed_control_value'] = momentum_metrics.apply(lambda row: teamed_control_index(row.team_name, row.game_home_team_name, row.game_away_team_name, row.smoothed_spread_of_ending_value_shot), axis = 1)

momentum_metrics_r = momentum_metrics.rename(columns = {'team_name':'Team','opponent_team_name':'Opposition'})

momentum_metrics_r['game_id_team_opp'] = momentum_metrics_r['game_id'].astype(str)+'_'+momentum_metrics_r['Team']+'_'+momentum_metrics_r['Opposition']

momentum_metrics_r.head()

Downloading: 100%|█████████████████████████████████████████████████████████████|


/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3138454860.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  momentum_metrics[['team_tier', 'against_tier']] = ( momentum_metrics.apply(lambda row: find_opponent_strength(row.team_name, row.opponent_team_name), axis=1) .apply(pd.Series) )
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3138454860.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  momentum_metrics[['team_tier', 'against_tier']] = ( momentum_metrics.apply(lam

,game_id,competition_name,seq,gameClock,seq_period,seq_time_length,game_description,Team,smoothed_spread_of_ending_value_shot,game_home_team_name,game_away_team_name,Opposition,team_tier,against_tier,teamed_control_value,game_id_team_opp
0,2602931,Under 18 Internationals,0a,0,1.0,4.0,England U18 - Uzbekistan U18,Uzbekistan U18,-0.004860,England U18,Uzbekistan U18,England U18,3,1,0.004860,2602931_Uzbekistan U18_England U18
1,2602931,Under 18 Internationals,1a,11,1.0,3.0,England U18 - Uzbekistan U18,England U18,-0.004954,England U18,Uzbekistan U18,Uzbekistan U18,1,3,-0.004954,2602931_England U18_Uzbekistan U18
2,2602931,Under 18 Internationals,2b,57,1.0,7.0,England U18 - Uzbekistan U18,Uzbekistan U18,-0.057810,England U18,Uzbekistan U18,England U18,3,1,0.057810,2602931_Uzbekistan U18_England U18
3,2602931,Under 18 Internationals,5a,71,1.0,14.0,England U18 - Uzbekistan U18,England U18,-0.067949,England U18,Uzbekistan U18,Uzbekistan U18,1,3,-0.067949,2602931_England U18_Uzbekistan U18
4,2602931,Under 18 Internationals,6a,88,1.0,1.0,England U18 - Uzbekistan U18,Uzbekistan U18,-0.063721,England U18,Uzbekistan U18,England U18,3,1,0.063721,2602931_Uzbekistan U18_England U18


In [532]:
momentum_metrics_r[momentum_metrics_r['Team'].str.contains('England U21')]['game_description'].unique()

array([], dtype=object)

In [533]:
momentum_metrics_r[momentum_metrics_r['teamed_control_value'] == 1000]

,game_id,competition_name,seq,gameClock,seq_period,seq_time_length,game_description,Team,smoothed_spread_of_ending_value_shot,game_home_team_name,game_away_team_name,Opposition,team_tier,against_tier,teamed_control_value,game_id_team_opp


In [534]:
momentum_metrics_r[(momentum_metrics_r['smoothed_spread_of_ending_value_shot'] > 0) & (momentum_metrics_r['Team']== momentum_metrics_r['game_home_team_name']) & (momentum_metrics_r['smoothed_spread_of_ending_value_shot'] < 0)]


,game_id,competition_name,seq,gameClock,seq_period,seq_time_length,game_description,Team,smoothed_spread_of_ending_value_shot,game_home_team_name,game_away_team_name,Opposition,team_tier,against_tier,teamed_control_value,game_id_team_opp


In [535]:
def add_momentum_label(value):

    if value >= 0.5:
        return 'High Control'
    elif value >= 0.2:
        return 'Control'
    elif value >= -0.2:
        return 'Balanced'
    elif value >= -0.5:
        return 'Opponent Control'
    elif value < -0.5:
        return 'Opponent High Control'
    else:
        return ''        

momentum_metrics_r['momentum_label'] = momentum_metrics_r.apply(lambda row: add_momentum_label(row.teamed_control_value), axis = 1)
momentum_metrics_r['game_seq_id'] = momentum_metrics_r['game_id'].astype(str)+'_'+momentum_metrics_r['seq']
momentum_metrics_r['game_id'] = momentum_metrics_r['game_id'].astype(int)

momentum_metrics_r.head()

,game_id,competition_name,seq,gameClock,seq_period,seq_time_length,game_description,Team,smoothed_spread_of_ending_value_shot,game_home_team_name,game_away_team_name,Opposition,team_tier,against_tier,teamed_control_value,game_id_team_opp,momentum_label,game_seq_id
0,2602931,Under 18 Internationals,0a,0,1.0,4.0,England U18 - Uzbekistan U18,Uzbekistan U18,-0.004860,England U18,Uzbekistan U18,England U18,3,1,0.004860,2602931_Uzbekistan U18_England U18,Balanced,2602931_0a
1,2602931,Under 18 Internationals,1a,11,1.0,3.0,England U18 - Uzbekistan U18,England U18,-0.004954,England U18,Uzbekistan U18,Uzbekistan U18,1,3,-0.004954,2602931_England U18_Uzbekistan U18,Balanced,2602931_1a
2,2602931,Under 18 Internationals,2b,57,1.0,7.0,England U18 - Uzbekistan U18,Uzbekistan U18,-0.057810,England U18,Uzbekistan U18,England U18,3,1,0.057810,2602931_Uzbekistan U18_England U18,Balanced,2602931_2b
3,2602931,Under 18 Internationals,5a,71,1.0,14.0,England U18 - Uzbekistan U18,England U18,-0.067949,England U18,Uzbekistan U18,Uzbekistan U18,1,3,-0.067949,2602931_England U18_Uzbekistan U18,Balanced,2602931_5a
4,2602931,Under 18 Internationals,6a,88,1.0,1.0,England U18 - Uzbekistan U18,Uzbekistan U18,-0.063721,England U18,Uzbekistan U18,England U18,3,1,0.063721,2602931_Uzbekistan U18_England U18,Balanced,2602931_6a


In [536]:
momentum_metrics_r.dtypes

game_id                                   int64
competition_name                         object
seq                                      object
gameClock                                 Int64
seq_period                              float64
seq_time_length                         float64
game_description                         object
Team                                     object
smoothed_spread_of_ending_value_shot    float64
game_home_team_name                      object
game_away_team_name                      object
Opposition                               object
team_tier                                 int64
against_tier                              int64
teamed_control_value                    float64
game_id_team_opp                         object
momentum_label                           object
game_seq_id                              object
dtype: object

In [537]:
merged_metrics.head()

,game_id,Player,Team,Opposition,xG,Open_Play_xG,Shots,Passes,Corners,Big_Chances,Opta_Passes_in_Own_Half,Possessions_Won_in_Opp_Half,Possessions_Won_in_Final_3rd,Possessions_Won_in_Mid_3rd,team_tier,against_tier
494,2609005,Kenny Nzogang,Austria U17,England U17,0.000000,0.000000,0,1,0,0,1,0,0,1,2,1
496,2609005,Jakob Pokorny,Austria U17,England U17,0.085657,0.048520,2,39,0,0,34,1,1,0,2,1
501,2609005,Vasilije Markovic,Austria U17,England U17,0.139317,0.139317,2,24,0,0,9,3,2,5,2,1
511,2609005,Luca Weinhandl,Austria U17,England U17,0.257424,0.257424,3,25,0,0,18,0,0,0,2,1
524,2609005,Rafael Feldinger,Austria U17,England U17,0.061668,0.000000,1,36,0,0,12,2,0,4,2,1


In [538]:
id_vars_to_use = ['game_id','Player','Team','against_tier','Opposition','team_tier']

value_vars_to_use = ['Shots', 'Passes', 'Corners', 'Big_Chances', 'xG','Open_Play_xG','Opta_Passes_in_Own_Half','Possessions_Won_in_Opp_Half','Possessions_Won_in_Final_3rd','Possessions_Won_in_Mid_3rd']

melted_events = pd.melt(merged_metrics, id_vars=id_vars_to_use, value_vars=value_vars_to_use, var_name='KPI', value_name='Value').fillna(0)
filtered_melted_events = melted_events[~melted_events['Team'].isnull()].drop_duplicates()

teamed_melted_kpis_pre = filtered_melted_events.copy()

teamed_melted_kpis_pre_2 = teamed_melted_kpis_pre[teamed_melted_kpis_pre['Team'].str.contains('England')]

if type(team_name) == str:
    teamed_melted_kpis_pre_2['Team'] = team_name
elif type(team_name) == list:
    teamed_melted_kpis_pre_2['Team'] = team_name[0]

n_of_matches = len(teamed_melted_kpis_pre_2['game_id'].unique())

teamed_melted_kpis_pre_f = teamed_melted_kpis_pre_2.groupby(by = ['Team','against_tier','KPI'])['Value'].sum().reset_index()

opp_teamed_melted_kpis_pre = filtered_melted_events.copy()

opp_teamed_melted_kpis_pre = opp_teamed_melted_kpis_pre[opp_teamed_melted_kpis_pre['Opposition'].str.contains('England')]

if type(team_name) == str:
    opp_teamed_melted_kpis_pre['Opposition'] = team_name
elif type(team_name) == list:
    opp_teamed_melted_kpis_pre['Opposition'] = team_name[0]

opp_teamed_melted_kpis_pre_f = opp_teamed_melted_kpis_pre.groupby(by = ['Opposition','team_tier','KPI'])['Value'].sum().reset_index()

shots_ = teamed_melted_kpis_pre_f[teamed_melted_kpis_pre_f['KPI'] == 'Shots']['Value'].sum()
passes_ = teamed_melted_kpis_pre_f[teamed_melted_kpis_pre_f['KPI'] == 'Passes']['Value'].sum()
corners_ = teamed_melted_kpis_pre_f[teamed_melted_kpis_pre_f['KPI'] == 'Corners']['Value'].sum()
big_chances = teamed_melted_kpis_pre_f[teamed_melted_kpis_pre_f['KPI'] == 'Big_Chances']['Value'].sum()

opp_shots_ = opp_teamed_melted_kpis_pre_f[opp_teamed_melted_kpis_pre_f['KPI'] == 'Shots']['Value'].sum()
opp_passes_ = opp_teamed_melted_kpis_pre_f[opp_teamed_melted_kpis_pre_f['KPI'] == 'Passes']['Value'].sum()
opp_corners_ = opp_teamed_melted_kpis_pre_f[opp_teamed_melted_kpis_pre_f['KPI'] == 'Corners']['Value'].sum()
opp_big_chances = opp_teamed_melted_kpis_pre_f[opp_teamed_melted_kpis_pre_f['KPI'] == 'Big_Chances']['Value'].sum()

xg_created = teamed_melted_kpis_pre_f[teamed_melted_kpis_pre_f['KPI'] == 'xG']['Value'].sum()
open_play_xg_created = teamed_melted_kpis_pre_f[teamed_melted_kpis_pre_f['KPI'] == 'Open_Play_xG']['Value'].sum()
set_play_xg_created = xg_created - open_play_xg_created

opp_xg_created = opp_teamed_melted_kpis_pre_f[opp_teamed_melted_kpis_pre_f['KPI'] == 'xG']['Value'].sum()
opp_open_play_xg_created = opp_teamed_melted_kpis_pre_f[opp_teamed_melted_kpis_pre_f['KPI'] == 'Open_Play_xG']['Value'].sum()
opp_set_play_xg_created = opp_xg_created - opp_open_play_xg_created

final_third_possessions_won = teamed_melted_kpis_pre_f[teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Final_3rd']['Value'].sum()
mid_third_possessions_won = teamed_melted_kpis_pre_f[teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Mid_3rd']['Value'].sum()

opp_final_third_possessions_won = opp_teamed_melted_kpis_pre_f[opp_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Final_3rd']['Value'].sum()
opp_mid_third_possessions_won = opp_teamed_melted_kpis_pre_f[opp_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Mid_3rd']['Value'].sum()

possessions_won_in_opp_half = teamed_melted_kpis_pre_f[teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Opp_Half']['Value'].sum()
own_half_passes = teamed_melted_kpis_pre_f[teamed_melted_kpis_pre_f['KPI'] == 'Opta_Passes_in_Own_Half']['Value'].sum()

opp_possessions_won_in_opp_half = opp_teamed_melted_kpis_pre_f[opp_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Opp_Half']['Value'].sum()
opp_own_half_passes = opp_teamed_melted_kpis_pre_f[opp_teamed_melted_kpis_pre_f['KPI'] == 'Opta_Passes_in_Own_Half']['Value'].sum()

time_high_block = aggr_phases_metrics[aggr_phases_metrics['KPI'] == '% IN HIGH BLOCK']['time_diff'].iloc[0]
time_mid_block = aggr_phases_metrics[aggr_phases_metrics['KPI'] == '% IN MID BLOCK']['time_diff'].iloc[0]
time_low_block = aggr_phases_metrics[aggr_phases_metrics['KPI'] == '% IN LOW BLOCK']['time_diff'].iloc[0]

time_build_phase = aggr_phases_metrics[aggr_phases_metrics['KPI'] == '% IN BUILD PHASE']['time_diff'].iloc[0]
time_mid_phase = aggr_phases_metrics[aggr_phases_metrics['KPI'] == '% IN CREATE PHASE']['time_diff'].iloc[0]
time_finish_phase = aggr_phases_metrics[aggr_phases_metrics['KPI'] == '% IN FINISH PHASE']['time_diff'].iloc[0]


/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/31502532.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teamed_melted_kpis_pre_2['Team'] = team_name


In [539]:
opp_teamed_melted_kpis_pre_f[opp_teamed_melted_kpis_pre_f['KPI'] == 'Opta_Passes_in_Own_Half']

,Opposition,team_tier,KPI,Value
3,England U17,2,Opta_Passes_in_Own_Half,162.0
13,England U17,3,Opta_Passes_in_Own_Half,833.0


In [540]:
merged_metrics[merged_metrics['Opposition'].str.contains('Ireland')]

,game_id,Player,Team,Opposition,xG,Open_Play_xG,Shots,Passes,Corners,Big_Chances,Opta_Passes_in_Own_Half,Possessions_Won_in_Opp_Half,Possessions_Won_in_Final_3rd,Possessions_Won_in_Mid_3rd,team_tier,against_tier


In [541]:
print('Possession (%): ')
possess = 100*(passes_ / (passes_ + opp_passes_))
print(passes_, opp_passes_, possess)

print('PPDA: ')
offensive_recovery_intensity = opp_own_half_passes / possessions_won_in_opp_half
print(opp_own_half_passes, possessions_won_in_opp_half, offensive_recovery_intensity)

print('PPDA Against: ')
opp_offensive_recovery_intensity = own_half_passes / opp_possessions_won_in_opp_half
print(own_half_passes, opp_possessions_won_in_opp_half, opp_offensive_recovery_intensity)

print('Efficacy High Block: ')
efficacy_high_block = final_third_possessions_won*(300/time_high_block)
print(final_third_possessions_won, time_high_block, efficacy_high_block)
print('Efficacy Mid Block: ')
efficacy_mid_block = mid_third_possessions_won*(300/time_mid_block)
print(mid_third_possessions_won, time_mid_block, efficacy_mid_block)

print('Opponent Efficacy in High Block: ')
opp_efficacy_high_block = opp_final_third_possessions_won*(300/time_build_phase)
print(opp_final_third_possessions_won, time_build_phase, opp_efficacy_high_block)
print('Opponent Efficacy in Mid Block: ')
opp_efficacy_mid_block = opp_mid_third_possessions_won*(300/time_mid_phase)
print(opp_mid_third_possessions_won, time_mid_phase, opp_efficacy_mid_block)

print('Offensive Efficacy: ')
offensive_efficacy = open_play_xg_created*(300/time_finish_phase)
print(open_play_xg_created, time_finish_phase, offensive_efficacy)

print('Defensive Efficacy: ')
defensive_efficacy = opp_open_play_xg_created*(300/time_low_block)
print(opp_open_play_xg_created, time_low_block, defensive_efficacy)

print('Field Tilt: ')
field_tilt = 100*(time_finish_phase/(time_finish_phase+time_low_block))
print(time_finish_phase, time_low_block, field_tilt)

print('xG: ')
print(xg_created, n_of_matches, xg_created / n_of_matches)

metrics_out = []

metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'Shots', n_of_matches, shots_, n_of_matches, shots_ / n_of_matches])
metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'Shots Against', n_of_matches, opp_shots_, n_of_matches, opp_shots_ / n_of_matches])

metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'Corners', n_of_matches, corners_, n_of_matches, corners_ / n_of_matches])
metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'Corners Against', n_of_matches, opp_corners_, n_of_matches, opp_corners_ / n_of_matches])

metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'Big Chances', n_of_matches,0, 0, big_chances / n_of_matches])
metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'Big Chances Against', n_of_matches,0, 0, opp_big_chances / n_of_matches])

metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'xG', n_of_matches, xg_created, n_of_matches, xg_created / n_of_matches])
metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'xG Against', n_of_matches, opp_xg_created, n_of_matches, opp_xg_created / n_of_matches])

metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'xG Open Play', n_of_matches, open_play_xg_created, n_of_matches, open_play_xg_created / n_of_matches])
metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'xG Open Play Against', n_of_matches, opp_open_play_xg_created, n_of_matches, opp_open_play_xg_created / n_of_matches])

metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'xG Set Play', n_of_matches, set_play_xg_created, n_of_matches, set_play_xg_created / n_of_matches])
metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'xG Set Play Against', n_of_matches, opp_set_play_xg_created, n_of_matches, opp_set_play_xg_created / n_of_matches])

metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'Possession %', n_of_matches, passes_, opp_passes_, possess])
metrics_out.append([add_team_name, 0, add_team_name +' Key Current', 'Possession % Against', n_of_matches, passes_, opp_passes_, 100-possess])
metrics_out.append([add_team_name, 0, add_team_name +' Advanced Current', 'PPDA',n_of_matches,opp_own_half_passes, possessions_won_in_opp_half, offensive_recovery_intensity])
metrics_out.append([add_team_name, 0, add_team_name +' Advanced Current', 'PPDA Against',n_of_matches,own_half_passes, opp_possessions_won_in_opp_half, opp_offensive_recovery_intensity])
metrics_out.append([add_team_name, 0, add_team_name +' Advanced Current', 'Efficacy High Block',n_of_matches,final_third_possessions_won, time_high_block, efficacy_high_block])
metrics_out.append([add_team_name, 0, add_team_name +' Advanced Current', 'Efficacy Mid Block',n_of_matches,mid_third_possessions_won, time_mid_block, efficacy_mid_block])
metrics_out.append([add_team_name, 0, add_team_name +' Advanced Current', 'Opponent Efficacy High Block',n_of_matches,opp_final_third_possessions_won, time_build_phase, opp_efficacy_high_block])
metrics_out.append([add_team_name, 0, add_team_name +' Advanced Current', 'Opponent Efficacy Mid Block',n_of_matches,opp_mid_third_possessions_won, time_mid_phase, opp_efficacy_mid_block])
metrics_out.append([add_team_name, 0, add_team_name +' Advanced Current', 'Offensive Efficacy',n_of_matches,open_play_xg_created, time_finish_phase, offensive_efficacy])
metrics_out.append([add_team_name, 0, add_team_name +' Advanced Current', 'Defensive Efficacy',n_of_matches,opp_open_play_xg_created, time_low_block, defensive_efficacy])
metrics_out.append([add_team_name, 0, add_team_name +' Advanced Current', 'Field Tilt',n_of_matches,time_finish_phase, time_low_block, field_tilt])


Possession (%): 
1975.0 1692.0 53.85874011453504
PPDA: 
995.0 113.0 8.805309734513274
PPDA Against: 
1070.0 55.0 19.454545454545453
Efficacy High Block: 
47.0 1695.0880000000018 8.318152213926348
Efficacy Mid Block: 
154.0 2018.864999999999 22.88414529946283
Opponent Efficacy in High Block: 
19.0 1983.164 2.874194973285114
Opponent Efficacy in Mid Block: 
97.0 3218.633000000001 9.041105338819303
Offensive Efficacy: 
8.583603670820594 2124.511999999999 1.2120812220623747
Defensive Efficacy: 
5.418377138674259 1620.8559999999995 1.0028732605501527
Field Tilt: 
2124.511999999999 1620.8559999999995 56.72371847038795
xG: 
12.138418223634362 5 2.4276836447268724


In [542]:
print(offensive_efficacy)

1.2120812220623747


In [543]:
benchmark_metrics[['team_tier', 'against_tier']] = ( benchmark_metrics.apply(lambda row: find_opponent_strength(row.Team, row.Opposition), axis=1) .apply(pd.Series) )

In [544]:
benchmark_metrics['game_id_team_opp'] = benchmark_metrics['game_id'].astype(str)+'_'+benchmark_metrics['Team']+'_'+benchmark_metrics['Opposition']


In [545]:
first_tier_benchmark_metrics = benchmark_metrics[benchmark_metrics['team_tier'] == 1]
opp_first_tier_benchmark_metrics = benchmark_metrics[benchmark_metrics['against_tier'] == 1]

first_tier_benchmark_metrics.groupby(by = ['team_tier'])['Open_Play_xG'].sum()
opp_first_tier_benchmark_metrics.groupby(by = ['against_tier'])['Open_Play_xG'].sum()

against_tier
1    139.130008
Name: Open_Play_xG, dtype: float64

In [546]:
england_benchmark_metrics = benchmark_metrics[benchmark_metrics['Team'].str.contains('England')]
opp_england_benchmark_metrics = benchmark_metrics[benchmark_metrics['Opposition'].str.contains('England')]


In [547]:
display_all(england_benchmark_metrics.head())

,game_id,Player,Team,Opposition,xG,Open_Play_xG,Shots,Passes,Corners,Big_Chances,Opta_Passes_in_Own_Half,Possessions_Won_in_Opp_Half,Possessions_Won_in_Final_3rd,Possessions_Won_in_Mid_3rd,team_tier,against_tier,game_id_team_opp
9,2366349,Curtis Jones,England U21,Japan U22,0.0,0.0,0,18,0,0,12,0,0,0,1,2,2366349_England U21_Japan U22
19,2366349,Levi Colwill,England U21,Japan U22,0.0,0.0,0,14,0,0,13,0,0,0,1,2,2366349_England U21_Japan U22
38,2366349,Luke Thomas,England U21,Japan U22,0.0,0.0,0,22,0,0,10,1,0,1,1,2,2366349_England U21_Japan U22
41,2366349,Taylor Harwood-Bellis,England U21,Japan U22,0.0,0.0,0,62,0,0,61,0,0,0,1,2,2366349_England U21_Japan U22
60,2366349,Morgan Gibbs-White,England U21,Japan U22,0.0,0.0,0,21,0,0,10,0,0,0,1,2,2366349_England U21_Japan U22


In [548]:
display_all(england_benchmark_metrics[['game_id','Player','Team','Opposition','team_tier','against_tier','Open_Play_xG']].head())

,game_id,Player,Team,Opposition,team_tier,against_tier,Open_Play_xG
9,2366349,Curtis Jones,England U21,Japan U22,1,2,0.0
19,2366349,Levi Colwill,England U21,Japan U22,1,2,0.0
38,2366349,Luke Thomas,England U21,Japan U22,1,2,0.0
41,2366349,Taylor Harwood-Bellis,England U21,Japan U22,1,2,0.0
60,2366349,Morgan Gibbs-White,England U21,Japan U22,1,2,0.0


In [549]:
eng_test_tier_metrics = eng_tier_team_metrics[eng_tier_team_metrics['team_tier'] == 1].groupby(by = ['team_tier','KPI','n_of_matches'])['time_diff'].sum().reset_index()
eng_test_tier_metrics['Value'] = (eng_test_tier_metrics['time_diff'] / eng_test_tier_metrics.groupby(by = ['team_tier','n_of_matches'])['time_diff'].transform('sum') * 100).round(2)
eng_test_tier_metrics_out = eng_test_tier_metrics[['KPI','time_diff','Value','n_of_matches']]

eng_test_tier_metrics_out

,KPI,time_diff,Value,n_of_matches
0,% IN BUILD PHASE,26063.709,15.43,57
1,% IN CREATE PHASE,44941.788,26.61,57
2,% IN FINISH PHASE,30275.683,17.93,57
3,% IN HIGH BLOCK,24398.594,14.45,57
4,% IN LOW BLOCK,16401.158,9.71,57
5,% IN MID BLOCK,26801.133,15.87,57


In [550]:
first_tier_team_metrics = tier_team_metrics[tier_team_metrics['team_tier'] == 1].groupby(by = ['team_tier','KPI','n_of_matches'])['time_diff'].sum().reset_index()
first_tier_team_metrics['Value'] = (first_tier_team_metrics['time_diff'] / first_tier_team_metrics.groupby(by = ['team_tier','n_of_matches'])['time_diff'].transform('sum') * 100).round(2)
first_tier_team_metrics_out = first_tier_team_metrics[['KPI','time_diff','Value','n_of_matches']]

first_tier_team_metrics_out

,KPI,time_diff,Value,n_of_matches
0,% IN BUILD PHASE,62699.795,14.61,153
1,% IN CREATE PHASE,111536.314,25.99,153
2,% IN FINISH PHASE,75936.908,17.69,153
3,% IN HIGH BLOCK,54293.556,12.65,153
4,% IN LOW BLOCK,47807.105,11.14,153
5,% IN MID BLOCK,76877.440,17.91,153


In [551]:
eng_first_tier_team_metrics = eng_tier_team_metrics[eng_tier_team_metrics['team_tier'] == 1].groupby(by = ['team_tier','KPI','n_of_matches'])['time_diff'].sum().reset_index()
eng_first_tier_team_metrics['Value'] = (eng_first_tier_team_metrics['time_diff'] / eng_first_tier_team_metrics.groupby(by = ['team_tier','n_of_matches'])['time_diff'].transform('sum') * 100).round(2)
eng_first_tier_team_metrics_out = eng_first_tier_team_metrics[['KPI','time_diff','Value','n_of_matches']]

eng_first_tier_team_metrics_out

,KPI,time_diff,Value,n_of_matches
0,% IN BUILD PHASE,26063.709,15.43,57
1,% IN CREATE PHASE,44941.788,26.61,57
2,% IN FINISH PHASE,30275.683,17.93,57
3,% IN HIGH BLOCK,24398.594,14.45,57
4,% IN LOW BLOCK,16401.158,9.71,57
5,% IN MID BLOCK,26801.133,15.87,57


In [552]:
id_vars_to_use = ['game_id','Player','Team','Opposition','game_id_team_opp','team_tier','against_tier']

value_vars_to_use = ['Shots', 'Passes', 'Corners', 'Big_Chances', 'xG','Open_Play_xG','Opta_Passes_in_Own_Half','Possessions_Won_in_Opp_Half','Possessions_Won_in_Final_3rd','Possessions_Won_in_Mid_3rd']

eng_melted_events = pd.melt(england_benchmark_metrics, id_vars=id_vars_to_use, value_vars=value_vars_to_use, var_name='KPI', value_name='Value').fillna(0)
eng_filtered_melted_events = eng_melted_events[~eng_melted_events['Team'].isnull()].drop_duplicates()

for against_tier_ in [0,1,2,3]:

    print('Tier :'+str(against_tier_))
    print(' ')
    
    if against_tier_ != 0:
    
        eng_teamed_melted_kpis_pre = eng_filtered_melted_events.copy()
        
        eng_teamed_melted_kpis_pre_2 = eng_teamed_melted_kpis_pre[(eng_teamed_melted_kpis_pre['Team'].str.contains('England')) & (eng_teamed_melted_kpis_pre['against_tier'] == against_tier_)]
        
        if type(team_name) == str:
            eng_teamed_melted_kpis_pre_2['Team'] = team_name
        elif type(team_name) == list:
            eng_teamed_melted_kpis_pre_2['Team'] = team_name[0]
        
        eng_n_of_matches = len(eng_teamed_melted_kpis_pre_2['game_id_team_opp'].unique())
        
        eng_teamed_melted_kpis_pre_f = eng_teamed_melted_kpis_pre_2.groupby(by = ['team_tier','KPI'])['Value'].sum().reset_index()
        
        eng_opp_teamed_melted_events = pd.melt(opp_england_benchmark_metrics, id_vars=id_vars_to_use, value_vars=value_vars_to_use, var_name='KPI', value_name='Value').fillna(0)
        eng_opp_teamed_melted_kpis_pre = eng_opp_teamed_melted_events[~eng_opp_teamed_melted_events['Team'].isnull()].drop_duplicates()
        
        eng_opp_teamed_melted_kpis_pre = eng_opp_teamed_melted_kpis_pre[(eng_opp_teamed_melted_kpis_pre['Opposition'].str.contains('England')) & (eng_opp_teamed_melted_kpis_pre['team_tier'] == against_tier_)]
    
        eng_opp_n_of_matches = len(eng_opp_teamed_melted_kpis_pre['game_id_team_opp'].unique())

        eng_tier_team_metrics_f = eng_tier_team_metrics[eng_tier_team_metrics['against_tier'] == against_tier_]
        
    else:
        
        eng_teamed_melted_kpis_pre = eng_filtered_melted_events.copy()
        
        eng_teamed_melted_kpis_pre_2 = eng_teamed_melted_kpis_pre[(eng_teamed_melted_kpis_pre['Team'].str.contains('England'))]
        
        if type(team_name) == str:
            eng_teamed_melted_kpis_pre_2['Team'] = team_name
        elif type(team_name) == list:
            eng_teamed_melted_kpis_pre_2['Team'] = team_name[0]
        
        eng_n_of_matches = len(eng_teamed_melted_kpis_pre_2['game_id_team_opp'].unique())
        
        eng_teamed_melted_kpis_pre_f = eng_teamed_melted_kpis_pre_2.groupby(by = ['team_tier','KPI'])['Value'].sum().reset_index()
        
        eng_opp_teamed_melted_events = pd.melt(opp_england_benchmark_metrics, id_vars=id_vars_to_use, value_vars=value_vars_to_use+['xG'], var_name='KPI', value_name='Value').fillna(0)
        eng_opp_teamed_melted_kpis_pre = eng_opp_teamed_melted_events[~eng_opp_teamed_melted_events['Team'].isnull()].drop_duplicates()
        
        eng_opp_teamed_melted_kpis_pre = eng_opp_teamed_melted_kpis_pre[(eng_opp_teamed_melted_kpis_pre['Opposition'].str.contains('England'))]
    
        eng_opp_n_of_matches = len(eng_opp_teamed_melted_kpis_pre['game_id_team_opp'].unique())

        eng_tier_team_metrics_f = eng_test_tier_metrics_out.copy()
        
    if type(team_name) == str:
        eng_opp_teamed_melted_kpis_pre['Opposition'] = team_name
    elif type(team_name) == list:
        eng_opp_teamed_melted_kpis_pre['Opposition'] = team_name[0]
    
    eng_opp_teamed_melted_kpis_pre_f = eng_opp_teamed_melted_kpis_pre.groupby(by = ['against_tier','KPI'])['Value'].sum().reset_index()
    
    eng_shots_ = eng_teamed_melted_kpis_pre_f[eng_teamed_melted_kpis_pre_f['KPI'] == 'Shots']['Value'].iloc[0]
    eng_passes_ = eng_teamed_melted_kpis_pre_f[eng_teamed_melted_kpis_pre_f['KPI'] == 'Passes']['Value'].iloc[0]
    eng_corners_ = eng_teamed_melted_kpis_pre_f[eng_teamed_melted_kpis_pre_f['KPI'] == 'Corners']['Value'].iloc[0]
    eng_big_chances = eng_teamed_melted_kpis_pre_f[eng_teamed_melted_kpis_pre_f['KPI'] == 'Big_Chances']['Value'].iloc[0]
    
    eng_opp_shots_ = eng_opp_teamed_melted_kpis_pre_f[eng_opp_teamed_melted_kpis_pre_f['KPI'] == 'Shots']['Value'].iloc[0]
    eng_opp_passes_ = eng_opp_teamed_melted_kpis_pre_f[eng_opp_teamed_melted_kpis_pre_f['KPI'] == 'Passes']['Value'].iloc[0]
    eng_opp_corners_ = eng_opp_teamed_melted_kpis_pre_f[eng_opp_teamed_melted_kpis_pre_f['KPI'] == 'Corners']['Value'].iloc[0]
    eng_opp_big_chances = eng_opp_teamed_melted_kpis_pre_f[eng_opp_teamed_melted_kpis_pre_f['KPI'] == 'Big_Chances']['Value'].iloc[0]
    
    eng_xg_created = eng_teamed_melted_kpis_pre_f[eng_teamed_melted_kpis_pre_f['KPI'] == 'xG']['Value'].iloc[0]
    eng_open_play_xg_created = eng_teamed_melted_kpis_pre_f[eng_teamed_melted_kpis_pre_f['KPI'] == 'Open_Play_xG']['Value'].iloc[0]
    eng_set_play_xg_created = eng_xg_created - eng_open_play_xg_created
    
    eng_opp_xg_created = eng_opp_teamed_melted_kpis_pre_f[eng_opp_teamed_melted_kpis_pre_f['KPI'] == 'xG']['Value'].iloc[0]
    eng_opp_open_play_xg_created = eng_opp_teamed_melted_kpis_pre_f[eng_opp_teamed_melted_kpis_pre_f['KPI'] == 'Open_Play_xG']['Value'].iloc[0]
    eng_opp_set_play_xg_created = eng_opp_xg_created - eng_opp_open_play_xg_created
    
    eng_final_third_possessions_won = eng_teamed_melted_kpis_pre_f[eng_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Final_3rd']['Value'].iloc[0]
    eng_mid_third_possessions_won = eng_teamed_melted_kpis_pre_f[eng_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Mid_3rd']['Value'].iloc[0]
    
    eng_opp_final_third_possessions_won = eng_opp_teamed_melted_kpis_pre_f[eng_opp_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Final_3rd']['Value'].iloc[0]
    eng_opp_mid_third_possessions_won = eng_opp_teamed_melted_kpis_pre_f[eng_opp_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Mid_3rd']['Value'].iloc[0]
    
    eng_possessions_won_in_opp_half = eng_teamed_melted_kpis_pre_f[eng_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Opp_Half']['Value'].iloc[0]
    eng_own_half_passes = eng_teamed_melted_kpis_pre_f[eng_teamed_melted_kpis_pre_f['KPI'] == 'Opta_Passes_in_Own_Half']['Value'].iloc[0]
    
    eng_opp_possessions_won_in_opp_half = eng_opp_teamed_melted_kpis_pre_f[eng_opp_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Opp_Half']['Value'].iloc[0]
    eng_opp_own_half_passes = eng_opp_teamed_melted_kpis_pre_f[eng_opp_teamed_melted_kpis_pre_f['KPI'] == 'Opta_Passes_in_Own_Half']['Value'].iloc[0]

    eng_time_high_block = eng_tier_team_metrics_f[eng_tier_team_metrics_f['KPI'] == '% IN HIGH BLOCK']['time_diff'].sum()
    eng_time_mid_block = eng_tier_team_metrics_f[eng_tier_team_metrics_f['KPI'] == '% IN MID BLOCK']['time_diff'].sum()
    eng_time_low_block = eng_tier_team_metrics_f[eng_tier_team_metrics_f['KPI'] == '% IN LOW BLOCK']['time_diff'].sum()
    
    eng_time_build_phase = eng_tier_team_metrics_f[eng_tier_team_metrics_f['KPI'] == '% IN BUILD PHASE']['time_diff'].sum()
    eng_time_mid_phase = eng_tier_team_metrics_f[eng_tier_team_metrics_f['KPI'] == '% IN CREATE PHASE']['time_diff'].sum()
    eng_time_finish_phase = eng_tier_team_metrics_f[eng_tier_team_metrics_f['KPI'] == '% IN FINISH PHASE']['time_diff'].sum()
    
    print('Possession (%): ')
    eng_possess = 100*(eng_passes_ / (eng_passes_ + eng_opp_passes_))
    print(eng_passes_, eng_opp_passes_, eng_possess)
    
    print('PPDA: ')
    eng_offensive_recovery_intensity = eng_opp_own_half_passes / eng_possessions_won_in_opp_half
    print(eng_opp_own_half_passes, eng_possessions_won_in_opp_half, eng_offensive_recovery_intensity)
    
    print('PPDA Against: ')
    eng_opp_offensive_recovery_intensity = eng_own_half_passes / eng_opp_possessions_won_in_opp_half
    print(eng_own_half_passes, eng_opp_possessions_won_in_opp_half, eng_opp_offensive_recovery_intensity)
    
    print('Efficacy High Block: ')
    eng_efficacy_high_block = eng_final_third_possessions_won*(300/eng_time_high_block)
    print(eng_final_third_possessions_won, eng_time_high_block, eng_efficacy_high_block)
    print('Efficacy Mid Block: ')
    eng_efficacy_mid_block = eng_mid_third_possessions_won*(300/eng_time_mid_block)
    print(eng_mid_third_possessions_won, eng_time_mid_block, eng_efficacy_mid_block)
    
    print('Opponent Efficacy in High Block: ')
    eng_opp_efficacy_high_block = eng_opp_final_third_possessions_won*(300/eng_time_build_phase)
    print(eng_opp_final_third_possessions_won, eng_time_build_phase, eng_opp_efficacy_high_block)
    print('Opponent Efficacy in Mid Block: ')
    eng_opp_efficacy_mid_block = eng_opp_mid_third_possessions_won*(300/eng_time_mid_phase)
    print(eng_opp_mid_third_possessions_won, eng_time_mid_phase, eng_opp_efficacy_mid_block)
    
    print('Offensive Efficacy: ')
    eng_offensive_efficacy = eng_open_play_xg_created*(300/eng_time_finish_phase)
    print(eng_open_play_xg_created, eng_time_finish_phase, eng_offensive_efficacy)
    
    print('Defensive Efficacy: ')
    eng_defensive_efficacy = eng_opp_open_play_xg_created*(300/eng_time_low_block)
    print(eng_opp_open_play_xg_created, eng_time_low_block, eng_defensive_efficacy)
    
    print('Field Tilt: ')
    eng_field_tilt = 100*(eng_time_finish_phase/(eng_time_finish_phase+eng_time_low_block))
    print(eng_time_finish_phase, eng_time_low_block, eng_field_tilt)
    
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'Shots', eng_n_of_matches, eng_shots_, eng_n_of_matches, eng_shots_ / eng_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'Shots Against', eng_n_of_matches, eng_opp_shots_, eng_opp_n_of_matches, eng_opp_shots_ / eng_opp_n_of_matches])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'Corners', eng_n_of_matches, eng_corners_, eng_n_of_matches, eng_corners_ / eng_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'Corners Against', eng_n_of_matches, eng_opp_corners_, eng_opp_n_of_matches, eng_opp_corners_ / eng_opp_n_of_matches])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'Big Chances', eng_n_of_matches, eng_big_chances, eng_n_of_matches, eng_big_chances / eng_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'Big Chances Against', eng_n_of_matches, eng_opp_big_chances, eng_opp_n_of_matches, eng_opp_big_chances / eng_opp_n_of_matches])

    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'xG', eng_n_of_matches, eng_xg_created, eng_n_of_matches, eng_xg_created / eng_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'xG Against', eng_n_of_matches, eng_opp_xg_created, eng_opp_n_of_matches, eng_opp_xg_created / eng_opp_n_of_matches])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'xG Open Play', eng_n_of_matches, eng_open_play_xg_created, eng_n_of_matches, eng_open_play_xg_created / eng_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'xG Open Play Against', eng_n_of_matches, eng_opp_open_play_xg_created, eng_opp_n_of_matches, eng_opp_open_play_xg_created / eng_opp_n_of_matches])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'xG Set Play', eng_n_of_matches, eng_set_play_xg_created, eng_n_of_matches, eng_set_play_xg_created / eng_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Benchmark', 'xG Set Play Against', eng_n_of_matches, eng_opp_set_play_xg_created, eng_opp_n_of_matches, eng_opp_set_play_xg_created / eng_opp_n_of_matches])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Key Benchmark', 'Possession %', eng_n_of_matches, eng_passes_, eng_opp_passes_, eng_possess])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Key Benchmark', 'Possession % Against', eng_n_of_matches, eng_opp_passes_, eng_passes_, 100-eng_possess])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Benchmark', 'PPDA', eng_n_of_matches,eng_opp_own_half_passes, eng_possessions_won_in_opp_half, eng_offensive_recovery_intensity])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Benchmark', 'PPDA Against', eng_n_of_matches,eng_own_half_passes, eng_opp_possessions_won_in_opp_half, eng_opp_offensive_recovery_intensity])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Benchmark', 'Efficacy High Block', eng_n_of_matches,eng_final_third_possessions_won, eng_time_high_block, eng_efficacy_high_block])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Benchmark', 'Efficacy Mid Block', eng_n_of_matches,eng_mid_third_possessions_won, eng_time_mid_block, eng_efficacy_mid_block])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Benchmark', 'Opponent Efficacy High Block', eng_n_of_matches,eng_opp_final_third_possessions_won, eng_time_build_phase, eng_opp_efficacy_high_block])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Benchmark', 'Opponent Efficacy Mid Block', eng_n_of_matches,eng_opp_mid_third_possessions_won, eng_time_mid_phase, eng_opp_efficacy_mid_block])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Benchmark', 'Offensive Efficacy', eng_n_of_matches,eng_open_play_xg_created, eng_time_finish_phase, eng_offensive_efficacy])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Benchmark', 'Defensive Efficacy', eng_n_of_matches,eng_opp_open_play_xg_created, eng_time_low_block, eng_defensive_efficacy])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Benchmark', 'Field Tilt', eng_n_of_matches,eng_time_finish_phase, eng_time_low_block, eng_field_tilt])


Tier :0
 
Possession (%): 
30459.0 23132.0 56.83603590155063
PPDA: 
14614.0 1406.0 10.39402560455192
PPDA Against: 
15727.0 1102.0 14.271324863883848
Efficacy High Block: 
568.0 24398.593999999997 6.984008996583984
Efficacy Mid Block: 
1863.0 26801.132999999987 20.85359600282571
Opponent Efficacy in High Block: 
425.0 26063.708999999995 4.891859404968035
Opponent Efficacy in Mid Block: 
1609.0 44941.788 10.74056065593118
Offensive Efficacy: 
76.76190792769194 30275.683000000005 0.7606293267870315
Defensive Efficacy: 
63.967107539996505 16401.158 1.1700473992140648
Field Tilt: 
30275.683000000005 16401.158 64.86232219528311
Tier :1
 
Possession (%): 
9692.0 9010.0 51.82333440273767
PPDA: 
5568.0 478.0 11.648535564853557
PPDA Against: 
5102.0 382.0 13.356020942408376
Efficacy High Block: 
188.0 9837.421999999995 5.733209371317001
Efficacy Mid Block: 
654.0 10864.505999999994 18.05880543487206
Opponent Efficacy in High Block: 
132.0 8388.299 4.720861762319154
Opponent Efficacy in Mid Bloc

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/1035460406.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eng_teamed_melted_kpis_pre_2['Team'] = team_name
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/1035460406.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eng_teamed_melted_kpis_pre_2['Team'] = team_name
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/1035460406.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [553]:
test_benchmark_momentum = momentum_metrics_r[['game_id','Team','team_tier','Opposition','game_id_team_opp','against_tier','game_seq_id','seq_time_length','momentum_label']].drop_duplicates()

momentum_time_length = test_benchmark_momentum.groupby(by = ['game_id','game_seq_id','Team','team_tier','game_id_team_opp','Opposition','against_tier','momentum_label'])['seq_time_length'].sum().reset_index()
momentum_time_length['Value'] = (momentum_time_length['seq_time_length'] / momentum_time_length.groupby(by = ['game_id','Team','team_tier','Opposition','against_tier'])['seq_time_length'].transform('sum') * 100).round(2)

len(momentum_time_length[(momentum_time_length['Team'].str.contains('England')) ]['game_id'].unique())

57

In [554]:
current_momentum_data = momentum_time_length[momentum_time_length['game_id'].isin(find_game_id_filt['matchId'].unique())]


In [555]:
england_benchmark_momentum_data = momentum_time_length[momentum_time_length['game_id'].isin(benchmark_metrics_games)]

print(len(england_benchmark_momentum_data[(england_benchmark_momentum_data['Opposition'].str.contains('England')) ]['game_id'].unique()))
print(len(england_benchmark_momentum_data[(england_benchmark_momentum_data['Team'].str.contains('England')) ]['game_id'].unique()))

57
57


In [556]:
tier_1_benchmark_momentum_data = momentum_time_length[momentum_time_length['team_tier'] == 1]
len(tier_1_benchmark_momentum_data['game_id'].unique())

123

In [557]:
against_tier_ = 0
team_or_opp = 'Team'


if against_tier_ != 0:

        grouping_vars = ['team_tier','against_tier','momentum_label']
        no_momentum_label_grouping_vars = ['team_tier','against_tier']
        
        eng_teamed_momentum_kpis_pre_2 = england_benchmark_momentum_data[
                (england_benchmark_momentum_data[team_or_opp].str.contains('England')) &
                (england_benchmark_momentum_data['against_tier'] == against_tier_)
            ]

else:
        print('ZERO')
        grouping_vars = ['team_tier','momentum_label']
        no_momentum_label_grouping_vars = ['team_tier']
                
        eng_teamed_momentum_kpis_pre_2 = england_benchmark_momentum_data[england_benchmark_momentum_data[team_or_opp].str.contains('England')]


display_all(eng_teamed_momentum_kpis_pre_2)

'''
momentum_games = eng_teamed_momentum_kpis_pre_2.groupby(by = no_momentum_label_grouping_vars)['game_id'].nunique().reset_index().rename(columns = {'game_id':'game_count'})

momentum_count_seq = eng_teamed_momentum_kpis_pre_2.groupby(by =grouping_vars)['game_seq_id'].nunique().reset_index().rename(columns = {'game_seq_id':'seq_count'})

momentum_time_length = eng_teamed_momentum_kpis_pre_2.groupby(by = grouping_vars)['seq_time_length'].sum().reset_index().rename(columns = {'seq_time_length':'total_seq_time_length'})

momentum_benchmark_metrics = pd.merge(momentum_count_seq, momentum_time_length, on = grouping_vars, how = 'left')

momentum_benchmark_metrics = pd.merge(momentum_benchmark_metrics, momentum_games, on = no_momentum_label_grouping_vars, how = 'left')

momentum_benchmark_metrics
'''

ZERO


,game_id,game_seq_id,Team,team_tier,game_id_team_opp,Opposition,against_tier,momentum_label,seq_time_length,Value
0,2200777,2200777_0a,England U18,1,2200777_England U18_Wales U18,Wales U18,3,Balanced,6.0,0.28
1,2200777,2200777_0b,England U18,1,2200777_England U18_Wales U18,Wales U18,3,Balanced,26.0,1.22
5,2200777,2200777_102b,England U18,1,2200777_England U18_Wales U18,Wales U18,3,Balanced,77.0,3.60
7,2200777,2200777_106a,England U18,1,2200777_England U18_Wales U18,Wales U18,3,Balanced,53.0,2.48
9,2200777,2200777_108a,England U18,1,2200777_England U18_Wales U18,Wales U18,3,Balanced,3.0,0.14
...,...,...,...,...,...,...,...,...,...,...
40996,2609005,2609005_95a,England U17,1,2609005_England U17_Austria U17,Austria U17,2,Opponent Control,3.0,0.11
41000,2609005,2609005_97a,England U17,1,2609005_England U17_Austria U17,Austria U17,2,Opponent Control,2.0,0.07
41001,2609005,2609005_97a,England U17,1,2609005_England U17_Austria U17,Austria U17,2,Opponent High Control,1.0,0.04
41006,2609005,2609005_99a,England U17,1,2609005_England U17_Austria U17,Austria U17,2,Opponent High Control,41.0,1.52


"\nmomentum_games = eng_teamed_momentum_kpis_pre_2.groupby(by = no_momentum_label_grouping_vars)['game_id'].nunique().reset_index().rename(columns = {'game_id':'game_count'})\n\nmomentum_count_seq = eng_teamed_momentum_kpis_pre_2.groupby(by =grouping_vars)['game_seq_id'].nunique().reset_index().rename(columns = {'game_seq_id':'seq_count'})\n\nmomentum_time_length = eng_teamed_momentum_kpis_pre_2.groupby(by = grouping_vars)['seq_time_length'].sum().reset_index().rename(columns = {'seq_time_length':'total_seq_time_length'})\n\nmomentum_benchmark_metrics = pd.merge(momentum_count_seq, momentum_time_length, on = grouping_vars, how = 'left')\n\nmomentum_benchmark_metrics = pd.merge(momentum_benchmark_metrics, momentum_games, on = no_momentum_label_grouping_vars, how = 'left')\n\nmomentum_benchmark_metrics\n"

In [558]:
england_benchmark_momentum_data.head()

,game_id,game_seq_id,Team,team_tier,game_id_team_opp,Opposition,against_tier,momentum_label,seq_time_length,Value
0,2200777,2200777_0a,England U18,1,2200777_England U18_Wales U18,Wales U18,3,Balanced,6.0,0.28
1,2200777,2200777_0b,England U18,1,2200777_England U18_Wales U18,Wales U18,3,Balanced,26.0,1.22
2,2200777,2200777_101a,Wales U18,3,2200777_Wales U18_England U18,England U18,1,Balanced,3.0,0.27
3,2200777,2200777_101b,Wales U18,3,2200777_Wales U18_England U18,England U18,1,Balanced,29.0,2.59
4,2200777,2200777_101c,Wales U18,3,2200777_Wales U18_England U18,England U18,1,Balanced,13.0,1.16


In [559]:

def momentum_summing_time_length(england_benchmark_momentum_data, against_tier_, team_or_opp):

    # team_or_opp = 'Team ' or 'Opposition'

    if against_tier_ != 0:

        grouping_vars = ['team_tier','against_tier','momentum_label']
        no_momentum_label_grouping_vars = ['team_tier','against_tier']
        
        eng_teamed_momentum_kpis_pre_2 = england_benchmark_momentum_data[
                (england_benchmark_momentum_data[team_or_opp].str.contains('England')) &
                (england_benchmark_momentum_data['against_tier'] == against_tier_)
            ]

    else:
        grouping_vars = ['team_tier','momentum_label']
        no_momentum_label_grouping_vars = ['team_tier']
                
        eng_teamed_momentum_kpis_pre_2 = england_benchmark_momentum_data[
                (england_benchmark_momentum_data[team_or_opp].str.contains('England'))
            ]
    
        
    momentum_games = eng_teamed_momentum_kpis_pre_2.groupby(by = no_momentum_label_grouping_vars)['game_id'].nunique().reset_index().rename(columns = {'game_id':'game_count'})
    
    momentum_count_seq = eng_teamed_momentum_kpis_pre_2.groupby(by =grouping_vars)['game_seq_id'].nunique().reset_index().rename(columns = {'game_seq_id':'seq_count'})
    
    momentum_time_length = eng_teamed_momentum_kpis_pre_2.groupby(by = grouping_vars)['seq_time_length'].sum().reset_index().rename(columns = {'seq_time_length':'total_seq_time_length'})
    
    momentum_benchmark_metrics = pd.merge(momentum_count_seq, momentum_time_length, on = grouping_vars, how = 'left')
    
    momentum_benchmark_metrics = pd.merge(momentum_benchmark_metrics, momentum_games, on = no_momentum_label_grouping_vars, how = 'left')

    momentum_benchmark_metrics['Team'] = add_team_name

    if against_tier_ != 0:
        momentum_benchmark_metrics['Value'] = (momentum_benchmark_metrics['total_seq_time_length'] / momentum_benchmark_metrics.groupby(by = ['team_tier','against_tier'])['total_seq_time_length'].transform('sum') * 100).round(2)
    else:
        momentum_benchmark_metrics['Value'] = (momentum_benchmark_metrics['total_seq_time_length'] / momentum_benchmark_metrics.groupby(by = ['team_tier'])['total_seq_time_length'].transform('sum') * 100).round(2)
        momentum_benchmark_metrics['against_tier'] = 0 

    return momentum_benchmark_metrics


england_benchmark_momentum_metrics = pd.DataFrame()

for against_tier_ in [0,1,2,3]:

    print('Tier :' + str(against_tier_))
    print(' ')
    
    preconcat = momentum_summing_time_length(england_benchmark_momentum_data, against_tier_,'Team')
    
    print(len(preconcat))
    
    england_benchmark_momentum_metrics = pd.concat([england_benchmark_momentum_metrics, preconcat], ignore_index = True)

'''
england_benchmark_momentum_metrics_zero = england_benchmark_momentum_metrics[england_benchmark_momentum_metrics['against_tier'] == 0].copy().drop_duplicates()

england_benchmark_momentum_metrics_zero['Value'] = (england_benchmark_momentum_metrics_zero['total_seq_time_length'] / england_benchmark_momentum_metrics.groupby(by = ['team_tier'])['total_seq_time_length'].transform('sum') * 100).round(2)

england_benchmark_momentum_metrics_more_than_1 = england_benchmark_momentum_metrics[england_benchmark_momentum_metrics['against_tier'] > 0].copy().drop_duplicates()

england_benchmark_momentum_metrics_more_than_1['Value'] = (england_benchmark_momentum_metrics_more_than_1['total_seq_time_length'] / england_benchmark_momentum_metrics_more_than_1.groupby(by = ['team_tier','against_tier'])['total_seq_time_length'].transform('sum') * 100).round(2)

display_all(england_benchmark_momentum_metrics_zero)
'''

england_benchmark_momentum_metrics

Tier :0
 
5
Tier :1
 
5
Tier :2
 
5
Tier :3
 
5


,team_tier,momentum_label,seq_count,total_seq_time_length,game_count,Team,Value,against_tier
0,1,Balanced,2944,59881.0,57,England U17,50.69,0
1,1,Control,907,18735.0,57,England U17,15.86,0
2,1,High Control,902,17375.0,57,England U17,14.71,0
3,1,Opponent Control,472,10504.0,57,England U17,8.89,0
4,1,Opponent High Control,613,11631.0,57,England U17,9.85,0
5,1,Balanced,1154,22076.0,20,England U17,58.24,1
6,1,Control,310,5906.0,20,England U17,15.58,1
7,1,High Control,164,2235.0,20,England U17,5.90,1
8,1,Opponent Control,174,3869.0,20,England U17,10.21,1
9,1,Opponent High Control,195,3816.0,20,England U17,10.07,1


In [560]:
current_momentum_data.head()

,game_id,game_seq_id,Team,team_tier,game_id_team_opp,Opposition,against_tier,momentum_label,seq_time_length,Value
23999,2556968,2556968_0a,Venezuela U17,3,2556968_Venezuela U17_England U17,England U17,1,Balanced,16.0,1.11
24000,2556968,2556968_100a,Venezuela U17,3,2556968_Venezuela U17_England U17,England U17,1,High Control,5.0,0.35
24001,2556968,2556968_101a,England U17,1,2556968_England U17_Venezuela U17,Venezuela U17,3,Opponent Control,6.0,0.27
24002,2556968,2556968_102b,Venezuela U17,3,2556968_Venezuela U17_England U17,England U17,1,Control,10.0,0.69
24003,2556968,2556968_103b,England U17,1,2556968_England U17_Venezuela U17,Venezuela U17,3,Opponent Control,191.0,8.68


In [561]:
current_benchmark_momentum_metrics = pd.DataFrame()

for against_tier_ in [0]:

    print('Tier :' + str(against_tier_))
    print(' ')
    
    preconcat = momentum_summing_time_length(current_momentum_data, against_tier_,'Team')
    
    print(len(preconcat))
    
    current_benchmark_momentum_metrics = pd.concat([current_benchmark_momentum_metrics, preconcat], ignore_index = True)

current_benchmark_momentum_metrics

Tier :0
 
5


,team_tier,momentum_label,seq_count,total_seq_time_length,game_count,Team,Value,against_tier
0,1,Balanced,186,4346.0,5,England U17,40.92,0
1,1,Control,81,1325.0,5,England U17,12.48,0
2,1,High Control,137,2455.0,5,England U17,23.12,0
3,1,Opponent Control,33,1003.0,5,England U17,9.44,0
4,1,Opponent High Control,74,1491.0,5,England U17,14.04,0


In [562]:
current_momentum_data.head()

,game_id,game_seq_id,Team,team_tier,game_id_team_opp,Opposition,against_tier,momentum_label,seq_time_length,Value
23999,2556968,2556968_0a,Venezuela U17,3,2556968_Venezuela U17_England U17,England U17,1,Balanced,16.0,1.11
24000,2556968,2556968_100a,Venezuela U17,3,2556968_Venezuela U17_England U17,England U17,1,High Control,5.0,0.35
24001,2556968,2556968_101a,England U17,1,2556968_England U17_Venezuela U17,Venezuela U17,3,Opponent Control,6.0,0.27
24002,2556968,2556968_102b,Venezuela U17,3,2556968_Venezuela U17_England U17,England U17,1,Control,10.0,0.69
24003,2556968,2556968_103b,England U17,1,2556968_England U17_Venezuela U17,Venezuela U17,3,Opponent Control,191.0,8.68


In [563]:
def momentum_summing_time_length_tier1(england_benchmark_momentum_data, against_tier_):

    # team_or_opp = 'Team ' or 'Opposition'

    if against_tier_ != 0:

        grouping_vars = ['team_tier','against_tier','momentum_label']
        no_momentum_label_grouping_vars = ['team_tier','against_tier']
        
        eng_teamed_momentum_kpis_pre_2 = england_benchmark_momentum_data[
                (england_benchmark_momentum_data['against_tier'] == against_tier_)
            ]

    else:
        grouping_vars = ['team_tier','momentum_label']
        no_momentum_label_grouping_vars = ['team_tier']
                
        eng_teamed_momentum_kpis_pre_2 = england_benchmark_momentum_data.copy()
        
        
    momentum_games = eng_teamed_momentum_kpis_pre_2.groupby(by = no_momentum_label_grouping_vars)['game_id'].nunique().reset_index().rename(columns = {'game_id':'game_count'})
    
    momentum_count_seq = eng_teamed_momentum_kpis_pre_2.groupby(by =grouping_vars)['game_seq_id'].nunique().reset_index().rename(columns = {'game_seq_id':'seq_count'})
    
    momentum_time_length = eng_teamed_momentum_kpis_pre_2.groupby(by = grouping_vars)['seq_time_length'].sum().reset_index().rename(columns = {'seq_time_length':'total_seq_time_length'})
    
    momentum_benchmark_metrics = pd.merge(momentum_count_seq, momentum_time_length, on = grouping_vars, how = 'left')
    
    momentum_benchmark_metrics = pd.merge(momentum_benchmark_metrics, momentum_games, on = no_momentum_label_grouping_vars, how = 'left')

    

    if against_tier_ != 0:
        momentum_benchmark_metrics['Value'] = (momentum_benchmark_metrics['total_seq_time_length'] / momentum_benchmark_metrics.groupby(by = ['team_tier','against_tier'])['total_seq_time_length'].transform('sum') * 100).round(2)
    else:
        momentum_benchmark_metrics['Value'] = (momentum_benchmark_metrics['total_seq_time_length'] / momentum_benchmark_metrics.groupby(by = ['team_tier'])['total_seq_time_length'].transform('sum') * 100).round(2)
        momentum_benchmark_metrics['against_tier'] = 0 

    return momentum_benchmark_metrics
    
tier_1_benchmark_momentum_metrics = pd.DataFrame()

for against_tier_ in [0,1,2,3]:

    print('Tier :' + str(against_tier_))
    print(' ')
    
    preconcat = momentum_summing_time_length_tier1(tier_1_benchmark_momentum_data, against_tier_)
    
    print(len(preconcat))
    
    tier_1_benchmark_momentum_metrics = pd.concat([tier_1_benchmark_momentum_metrics, preconcat], ignore_index = True)

tier_1_benchmark_momentum_metrics

Tier :0
 
5
Tier :1
 
5
Tier :2
 
5
Tier :3
 
5


,team_tier,momentum_label,seq_count,total_seq_time_length,game_count,Value,against_tier
0,1,Balanced,7578,146149.0,123,47.67,0
1,1,Control,2344,47572.0,123,15.52,0
2,1,High Control,2894,59621.0,123,19.45,0
3,1,Opponent Control,1325,27048.0,123,8.82,0
4,1,Opponent High Control,1377,26183.0,123,8.54,0
5,1,Balanced,3331,59139.0,30,54.60,1
6,1,Control,764,13907.0,30,12.84,1
7,1,High Control,596,9849.0,30,9.09,1
8,1,Opponent Control,723,13689.0,30,12.64,1
9,1,Opponent High Control,595,11731.0,30,10.83,1


In [564]:
momentum_metrics_out_bq = pd.merge(tier_1_benchmark_momentum_metrics.rename(columns = {'Value':'Comp_Value'}).drop(columns = {'total_seq_time_length','seq_count','game_count'}), england_benchmark_momentum_metrics.rename(columns = {'Value':'Benchmark_Value'}).drop(columns = {'total_seq_time_length','seq_count','game_count'}), on = ['team_tier','against_tier','momentum_label'], how = 'left')
momentum_metrics_out_bq = pd.merge(momentum_metrics_out_bq, current_benchmark_momentum_metrics.drop(columns = {'total_seq_time_length','seq_count','game_count'}), on = ['Team','team_tier','against_tier','momentum_label'], how = 'left').fillna(0.0)

momentum_metrics_out_bq = momentum_metrics_out_bq.rename(columns = {'momentum_label':'KPI'})

momentum_metrics_out_bq_positive = momentum_metrics_out_bq[momentum_metrics_out_bq['KPI'].isin(['Control','High Control','Balanced'])]
momentum_metrics_out_bq_negative = momentum_metrics_out_bq[momentum_metrics_out_bq['KPI'].isin(['Opponent Control','Opponent High Control'])]

momentum_metrics_out_bq_positive['Value_norm_to_tier'] = (momentum_metrics_out_bq_positive['Value'] - momentum_metrics_out_bq_positive['Comp_Value'])/ momentum_metrics_out_bq_positive['Comp_Value']
momentum_metrics_out_bq_negative['Value_norm_to_tier'] = (momentum_metrics_out_bq_negative['Comp_Value'] - momentum_metrics_out_bq_negative['Value'])/ momentum_metrics_out_bq_negative['Comp_Value']

momentum_metrics_out_bq_positive['Value_norm_to_eng_benchmark'] = (momentum_metrics_out_bq_positive['Value'] - momentum_metrics_out_bq_positive['Benchmark_Value'])/ momentum_metrics_out_bq_positive['Benchmark_Value']
momentum_metrics_out_bq_negative['Value_norm_to_eng_benchmark'] = (momentum_metrics_out_bq_negative['Benchmark_Value'] - momentum_metrics_out_bq_negative['Value'])/ momentum_metrics_out_bq_negative['Benchmark_Value']

momentum_metrics_out_jjj = pd.concat([momentum_metrics_out_bq_positive, momentum_metrics_out_bq_negative], ignore_index = True)

momentum_metrics_out_jjj['Opposition'] = 'Aggregate'
momentum_metrics_out_jjj['game_id'] = 99999

display_all(momentum_metrics_out_jjj)

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3610739987.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  momentum_metrics_out_bq_positive['Value_norm_to_tier'] = (momentum_metrics_out_bq_positive['Value'] - momentum_metrics_out_bq_positive['Comp_Value'])/ momentum_metrics_out_bq_positive['Comp_Value']
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3610739987.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  momentum_metrics_out_bq_negative['Value_norm_to_tier'] = (m

,team_tier,KPI,Comp_Value,against_tier,Team,Benchmark_Value,Value,Value_norm_to_tier,Value_norm_to_eng_benchmark,Opposition,game_id
0,1,Balanced,47.67,0,England U17,50.69,40.92,-0.141598,-0.192740,Aggregate,99999
1,1,Control,15.52,0,England U17,15.86,12.48,-0.195876,-0.213115,Aggregate,99999
2,1,High Control,19.45,0,England U17,14.71,23.12,0.188689,0.571720,Aggregate,99999
3,1,Balanced,54.60,1,England U17,58.24,0.00,-1.000000,-1.000000,Aggregate,99999
4,1,Control,12.84,1,England U17,15.58,0.00,-1.000000,-1.000000,Aggregate,99999
5,1,High Control,9.09,1,England U17,5.90,0.00,-1.000000,-1.000000,Aggregate,99999
6,1,Balanced,46.91,2,England U17,48.32,0.00,-1.000000,-1.000000,Aggregate,99999
7,1,Control,17.04,2,England U17,13.67,0.00,-1.000000,-1.000000,Aggregate,99999
8,1,High Control,16.45,2,England U17,13.43,0.00,-1.000000,-1.000000,Aggregate,99999
9,1,Balanced,41.96,3,England U17,45.71,0.00,-1.000000,-1.000000,Aggregate,99999


In [565]:
pre_teamed_current_momentum_data = current_momentum_data[current_momentum_data['Team'].str.contains('England')]

opp_teamed_current_momentum_data = current_momentum_data[~current_momentum_data['Team'].str.contains('England')]

swap_map = {
    "Control": "Opponent Control",
    "Opponent Control": "Control",
    "High Control": "Opponent High Control",
    "Opponent High Control": "High Control"
}

opp_teamed_current_momentum_data["momentum_label"] = (
    opp_teamed_current_momentum_data["momentum_label"].map(swap_map)
)

opp_teamed_current_momentum_data_r = opp_teamed_current_momentum_data.rename(columns = {'Team':'Opp','team_tier':'Opp1'})
opp_teamed_current_momentum_data_r2 = opp_teamed_current_momentum_data_r.rename(columns = {'Opposition':'Team','against_tier':'team_tier'})
opp_teamed_current_momentum_data_r3 = opp_teamed_current_momentum_data_r2.rename(columns = {'Opp':'Opposition','Opp1':'against_tier'})

teamed_current_momentum_data = pd.concat([pre_teamed_current_momentum_data, opp_teamed_current_momentum_data_r3[pre_teamed_current_momentum_data.columns]], ignore_index = True)

teamed_current_momentum_data.groupby(by = ['Team']).count()

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/1084911200.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opp_teamed_current_momentum_data["momentum_label"] = (


,game_id,game_seq_id,team_tier,game_id_team_opp,Opposition,against_tier,momentum_label,seq_time_length,Value
Team,,,,,,,,,
England U17,1020,1020,1020,1020,1020,1020,863,1020,1020


In [566]:
teamed_current_momentum_data.head()

,game_id,game_seq_id,Team,team_tier,game_id_team_opp,Opposition,against_tier,momentum_label,seq_time_length,Value
0,2556968,2556968_101a,England U17,1,2556968_England U17_Venezuela U17,Venezuela U17,3,Opponent Control,6.0,0.27
1,2556968,2556968_103b,England U17,1,2556968_England U17_Venezuela U17,Venezuela U17,3,Opponent Control,191.0,8.68
2,2556968,2556968_105a,England U17,1,2556968_England U17_Venezuela U17,Venezuela U17,3,Opponent Control,21.0,0.95
3,2556968,2556968_105b,England U17,1,2556968_England U17_Venezuela U17,Venezuela U17,3,Opponent Control,20.0,0.91
4,2556968,2556968_105c,England U17,1,2556968_England U17_Venezuela U17,Venezuela U17,3,Opponent Control,124.0,5.64


In [567]:
momentum_current_count_seq = teamed_current_momentum_data.groupby(by =['game_id','Team','Opposition','team_tier','against_tier','momentum_label'])['game_seq_id'].nunique().reset_index().rename(columns = {'game_seq_id':'seq_count'})
    
momentum_current_time_length = teamed_current_momentum_data.groupby(by = ['game_id','Team','Opposition','team_tier','against_tier','momentum_label'])['seq_time_length'].sum().reset_index().rename(columns = {'seq_time_length':'total_seq_time_length'})
    
momentum_current_benchmark_metrics = pd.merge(momentum_current_count_seq, momentum_current_time_length, on = ['game_id','Team','Opposition','team_tier','against_tier','momentum_label'], how = 'left')
    
momentum_current_benchmark_metrics['Value'] = (momentum_current_benchmark_metrics['total_seq_time_length'] / momentum_current_benchmark_metrics.groupby(by = ['game_id','Team','Opposition','team_tier','against_tier'])['total_seq_time_length'].transform('sum') * 100).round(2)

momentum_current_benchmark_metrics.head()

,game_id,Team,Opposition,team_tier,against_tier,momentum_label,seq_count,total_seq_time_length,Value
0,2556968,England U17,Venezuela U17,1,3,Balanced,46,928.0,28.86
1,2556968,England U17,Venezuela U17,1,3,Control,13,112.0,3.48
2,2556968,England U17,Venezuela U17,1,3,Opponent Control,33,964.0,29.98
3,2556968,England U17,Venezuela U17,1,3,Opponent High Control,64,1212.0,37.69
4,2556970,England U17,Haiti U17,1,3,Balanced,17,356.0,9.56


In [568]:
agg_momentum_metrics_out_bq_pre = pd.merge(momentum_current_benchmark_metrics, tier_1_benchmark_momentum_metrics.rename(columns = {'Value':'Comp_Value'}).drop(columns = {'total_seq_time_length','seq_count','game_count'}), on = ['team_tier','against_tier','momentum_label'], how = 'left').fillna(0.0)

agg_momentum_metrics_out_bq = pd.merge(agg_momentum_metrics_out_bq_pre, england_benchmark_momentum_metrics.rename(columns = {'Value':'Benchmark_Value'}).drop(columns = {'total_seq_time_length','seq_count','game_count'}), on = ['team_tier','against_tier','momentum_label','Team'], how = 'left')

agg_momentum_metrics_out_bq = agg_momentum_metrics_out_bq.rename(columns = {'momentum_label':'KPI'})

agg_momentum_metrics_out_bq_positive = agg_momentum_metrics_out_bq[agg_momentum_metrics_out_bq['KPI'].isin(['Control','High Control','Balanced'])]
agg_momentum_metrics_out_bq_negative = agg_momentum_metrics_out_bq[agg_momentum_metrics_out_bq['KPI'].isin(['Opponent Control','Opponent High Control'])]

agg_momentum_metrics_out_bq_positive['Value_norm_to_tier'] = (agg_momentum_metrics_out_bq_positive['Value'] - agg_momentum_metrics_out_bq_positive['Comp_Value'])/ agg_momentum_metrics_out_bq_positive['Comp_Value']
agg_momentum_metrics_out_bq_negative['Value_norm_to_tier'] = (agg_momentum_metrics_out_bq_negative['Comp_Value'] - agg_momentum_metrics_out_bq_negative['Value'])/ agg_momentum_metrics_out_bq_negative['Comp_Value']

agg_momentum_metrics_out_bq_positive['Value_norm_to_eng_benchmark'] = (agg_momentum_metrics_out_bq_positive['Value'] - agg_momentum_metrics_out_bq_positive['Benchmark_Value'])/ agg_momentum_metrics_out_bq_positive['Benchmark_Value']
agg_momentum_metrics_out_bq_negative['Value_norm_to_eng_benchmark'] = (agg_momentum_metrics_out_bq_negative['Benchmark_Value'] - agg_momentum_metrics_out_bq_negative['Value'])/ agg_momentum_metrics_out_bq_negative['Benchmark_Value']

agg_momentum_metrics_out_jjj = pd.concat([agg_momentum_metrics_out_bq_positive, agg_momentum_metrics_out_bq_negative], ignore_index = True)
add_date = find_game_id_filt[['matchId','matchDateTime']].rename(columns = {'matchId':'game_id'})

agg_momentum_metrics_out_jjj_fff = pd.merge(agg_momentum_metrics_out_jjj, add_date, on = ['game_id'], how='left')
agg_momentum_metrics_out_jjj_fff['game_description'] = agg_momentum_metrics_out_jjj_fff['Team'] +'-' + agg_momentum_metrics_out_jjj_fff['Opposition'] 

agg_momentum_metrics_out_jjj_fff.sort_values(by = ['game_id'])

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/1057406102.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_momentum_metrics_out_bq_positive['Value_norm_to_tier'] = (agg_momentum_metrics_out_bq_positive['Value'] - agg_momentum_metrics_out_bq_positive['Comp_Value'])/ agg_momentum_metrics_out_bq_positive['Comp_Value']
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/1057406102.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_momentum_metrics_out_bq_negative['Valu

,game_id,Team,Opposition,team_tier,against_tier,KPI,seq_count,total_seq_time_length,Value,Comp_Value,Benchmark_Value,Value_norm_to_tier,Value_norm_to_eng_benchmark,matchDateTime,game_description
0,2556968,England U17,Venezuela U17,1,3,Balanced,46,928.0,28.86,41.96,45.71,-0.312202,-0.368628,2025-11-04,England U17-Venezuela U17
1,2556968,England U17,Venezuela U17,1,3,Control,13,112.0,3.48,16.94,18.76,-0.794569,-0.814499,2025-11-04,England U17-Venezuela U17
14,2556968,England U17,Venezuela U17,1,3,Opponent High Control,64,1212.0,37.69,4.39,3.76,-7.585421,-9.023936,2025-11-04,England U17-Venezuela U17
13,2556968,England U17,Venezuela U17,1,3,Opponent Control,33,964.0,29.98,6.07,6.43,-3.939044,-3.662519,2025-11-04,England U17-Venezuela U17
2,2556970,England U17,Haiti U17,1,3,Balanced,17,356.0,9.56,41.96,45.71,-0.772164,-0.790855,2025-11-07,England U17-Haiti U17
3,2556970,England U17,Haiti U17,1,3,Control,70,1234.0,33.13,16.94,18.76,0.955726,0.765991,2025-11-07,England U17-Haiti U17
4,2556970,England U17,Haiti U17,1,3,High Control,104,2135.0,57.32,30.64,25.35,0.870757,1.261144,2025-11-07,England U17-Haiti U17
5,2556972,England U17,Egypt U17,1,3,Balanced,38,1195.0,44.14,41.96,45.71,0.051954,-0.034347,2025-11-10,England U17-Egypt U17
6,2556972,England U17,Egypt U17,1,3,Control,51,627.0,23.16,16.94,18.76,0.367178,0.234542,2025-11-10,England U17-Egypt U17
7,2556972,England U17,Egypt U17,1,3,High Control,66,885.0,32.69,30.64,25.35,0.066906,0.289546,2025-11-10,England U17-Egypt U17


In [569]:
id_vars_to_use = ['game_id','Player','Team','Opposition','game_id_team_opp','team_tier','against_tier']

value_vars_to_use = ['Shots', 'Passes', 'Corners', 'Big_Chances', 'xG','Open_Play_xG','Opta_Passes_in_Own_Half','Possessions_Won_in_Opp_Half','Possessions_Won_in_Final_3rd','Possessions_Won_in_Mid_3rd']

for against_tier_ in [0,1,2,3]:

    print('Tier: '+str(against_tier_))
    print(' ')
    
    if against_tier_ != 0:
    
        first_tier_benchmark_metrics_j = first_tier_benchmark_metrics[first_tier_benchmark_metrics['against_tier'] == against_tier_]
        opp_first_tier_benchmark_metrics_j = opp_first_tier_benchmark_metrics[opp_first_tier_benchmark_metrics['team_tier'] == against_tier_]
        first_tier_team_metrics = tier_team_metrics[ (tier_team_metrics['team_tier'] == 1) & (tier_team_metrics['against_tier'] == against_tier_) ]
    else:
        first_tier_benchmark_metrics_j = first_tier_benchmark_metrics.copy()
        opp_first_tier_benchmark_metrics_j = opp_first_tier_benchmark_metrics.copy()
        first_tier_team_metrics = first_tier_team_metrics_out.copy()
        
    benchmark_melted_events = pd.melt(first_tier_benchmark_metrics_j, id_vars=id_vars_to_use, value_vars=value_vars_to_use, var_name='KPI', value_name='Value').fillna(0)
    benchmark_teamed_melted_kpis_pre = benchmark_melted_events[~benchmark_melted_events['Team'].isnull()].drop_duplicates()
    
    benchmark_opp_teamed_events = pd.melt(opp_first_tier_benchmark_metrics_j, id_vars=id_vars_to_use, value_vars=value_vars_to_use, var_name='KPI', value_name='Value').fillna(0)
    benchmark_opp_teamed_melted_kpis_pre = benchmark_opp_teamed_events[~benchmark_opp_teamed_events['Team'].isnull()].drop_duplicates()

    b_n_of_matches = len(benchmark_teamed_melted_kpis_pre['game_id_team_opp'].unique())
    opp_b_n_of_matches = len(opp_first_tier_benchmark_metrics_j['game_id_team_opp'].unique())
    
    benchmark_teamed_melted_kpis_pre_f = benchmark_teamed_melted_kpis_pre.groupby(by = ['team_tier','KPI'])['Value'].sum().reset_index()
    
    benchmark_opp_teamed_melted_kpis_pre_f = benchmark_opp_teamed_melted_kpis_pre.groupby(by = ['against_tier','KPI'])['Value'].sum().reset_index()
    
    b_shots_ = benchmark_teamed_melted_kpis_pre_f[benchmark_teamed_melted_kpis_pre_f['KPI'] == 'Shots']['Value'].iloc[0]
    b_passes_ = benchmark_teamed_melted_kpis_pre_f[benchmark_teamed_melted_kpis_pre_f['KPI'] == 'Passes']['Value'].iloc[0]
    b_corners_ = benchmark_teamed_melted_kpis_pre_f[benchmark_teamed_melted_kpis_pre_f['KPI'] == 'Corners']['Value'].iloc[0]
    b_big_chances = benchmark_teamed_melted_kpis_pre_f[benchmark_teamed_melted_kpis_pre_f['KPI'] == 'Big_Chances']['Value'].iloc[0]
    
    b_opp_shots_ = benchmark_opp_teamed_melted_kpis_pre_f[benchmark_opp_teamed_melted_kpis_pre_f['KPI'] == 'Shots']['Value'].iloc[0]
    b_opp_passes_ = benchmark_opp_teamed_melted_kpis_pre_f[benchmark_opp_teamed_melted_kpis_pre_f['KPI'] == 'Passes']['Value'].iloc[0]
    b_opp_corners_ = benchmark_opp_teamed_melted_kpis_pre_f[benchmark_opp_teamed_melted_kpis_pre_f['KPI'] == 'Corners']['Value'].iloc[0]
    b_opp_big_chances = benchmark_opp_teamed_melted_kpis_pre_f[benchmark_opp_teamed_melted_kpis_pre_f['KPI'] == 'Big_Chances']['Value'].iloc[0]
    
    b_xg_created = benchmark_teamed_melted_kpis_pre_f[benchmark_teamed_melted_kpis_pre_f['KPI'] == 'xG']['Value'].sum()
    b_open_play_xg_created = benchmark_teamed_melted_kpis_pre_f[benchmark_teamed_melted_kpis_pre_f['KPI'] == 'Open_Play_xG']['Value'].sum()
    b_set_play_xg_created = b_xg_created - b_open_play_xg_created
    
    b_opp_xg_created = benchmark_opp_teamed_melted_kpis_pre_f[benchmark_opp_teamed_melted_kpis_pre_f['KPI'] == 'xG']['Value'].sum()
    b_opp_open_play_xg_created = benchmark_opp_teamed_melted_kpis_pre_f[benchmark_opp_teamed_melted_kpis_pre_f['KPI'] == 'Open_Play_xG']['Value'].sum()
    b_opp_set_play_xg_created = b_opp_xg_created - b_opp_open_play_xg_created
    
    b_final_third_possessions_won = benchmark_teamed_melted_kpis_pre_f[benchmark_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Final_3rd']['Value'].sum()
    b_mid_third_possessions_won = benchmark_teamed_melted_kpis_pre_f[benchmark_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Mid_3rd']['Value'].sum()
    
    b_opp_final_third_possessions_won = benchmark_opp_teamed_melted_kpis_pre_f[benchmark_opp_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Final_3rd']['Value'].sum()
    b_opp_mid_third_possessions_won = benchmark_opp_teamed_melted_kpis_pre_f[benchmark_opp_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Mid_3rd']['Value'].sum()
    
    b_possessions_won_in_opp_half = benchmark_teamed_melted_kpis_pre_f[benchmark_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Opp_Half']['Value'].sum()
    b_own_half_passes = benchmark_teamed_melted_kpis_pre_f[benchmark_teamed_melted_kpis_pre_f['KPI'] == 'Opta_Passes_in_Own_Half']['Value'].sum()
    
    b_opp_possessions_won_in_opp_half = benchmark_opp_teamed_melted_kpis_pre_f[benchmark_opp_teamed_melted_kpis_pre_f['KPI'] == 'Possessions_Won_in_Opp_Half']['Value'].sum()
    b_opp_own_half_passes = benchmark_opp_teamed_melted_kpis_pre_f[benchmark_opp_teamed_melted_kpis_pre_f['KPI'] == 'Opta_Passes_in_Own_Half']['Value'].sum()
    
    b_time_high_block = first_tier_team_metrics[(first_tier_team_metrics['KPI'] == '% IN HIGH BLOCK')]['time_diff'].sum()
    b_time_mid_block = first_tier_team_metrics[(first_tier_team_metrics['KPI'] == '% IN MID BLOCK')]['time_diff'].sum()
    b_time_low_block = first_tier_team_metrics[(first_tier_team_metrics['KPI'] == '% IN LOW BLOCK')]['time_diff'].sum()
    
    b_time_build_phase = first_tier_team_metrics[(first_tier_team_metrics['KPI'] == '% IN BUILD PHASE')]['time_diff'].sum()
    b_time_mid_phase = first_tier_team_metrics[(first_tier_team_metrics['KPI'] == '% IN CREATE PHASE')]['time_diff'].sum()
    b_time_finish_phase = first_tier_team_metrics[(first_tier_team_metrics['KPI'] == '% IN FINISH PHASE')]['time_diff'].sum()
    
    print('Possession (%): ')
    b_possess = 100*(b_passes_ / (b_passes_ + b_opp_passes_))
    print(b_passes_, b_opp_passes_, b_possess)
    
    print('PPDA: ')
    b_offensive_recovery_intensity = b_opp_own_half_passes / b_possessions_won_in_opp_half
    print(b_opp_own_half_passes, b_possessions_won_in_opp_half, b_offensive_recovery_intensity)
    
    print('PPDA Against: ')
    b_opp_offensive_recovery_intensity = b_own_half_passes / b_opp_possessions_won_in_opp_half
    print(b_own_half_passes, b_opp_possessions_won_in_opp_half, b_opp_offensive_recovery_intensity)
    
    print('Efficacy High Block: ')
    b_efficacy_high_block = b_final_third_possessions_won*(300/b_time_high_block)
    print(b_final_third_possessions_won, b_time_high_block, b_efficacy_high_block)
    print('Efficacy Mid Block: ')
    b_efficacy_mid_block = b_mid_third_possessions_won*(300/b_time_mid_block)
    print(b_mid_third_possessions_won, b_time_mid_block, b_efficacy_mid_block)
    
    print('Opponent Efficacy in High Block: ')
    b_opp_efficacy_high_block = b_opp_final_third_possessions_won*(300/b_time_build_phase)
    print(b_opp_final_third_possessions_won, b_time_build_phase, b_opp_efficacy_high_block)
    print('Opponent Efficacy in Mid Block: ')
    b_opp_efficacy_mid_block = b_opp_mid_third_possessions_won*(300/b_time_mid_phase)
    print(b_opp_mid_third_possessions_won, b_time_mid_phase, b_opp_efficacy_mid_block)
    
    print('Offensive Efficacy: ')
    b_offensive_efficacy = b_open_play_xg_created*(300/b_time_finish_phase)
    print(b_open_play_xg_created, b_time_finish_phase, b_offensive_efficacy)
    
    print('Defensive Efficacy: ')
    b_defensive_efficacy = b_opp_open_play_xg_created*(300/b_time_low_block)
    print(b_opp_open_play_xg_created, b_time_low_block, b_defensive_efficacy)
    
    print('Field Tilt: ')
    b_field_tilt = 100*(b_time_finish_phase/(b_time_finish_phase+b_time_low_block))
    print(b_time_finish_phase, b_time_low_block, b_field_tilt)
    
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'Shots', b_n_of_matches, b_shots_, b_n_of_matches, b_shots_ / b_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'Shots Against', b_n_of_matches, b_opp_shots_, opp_b_n_of_matches, b_opp_shots_ / opp_b_n_of_matches])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'Corners', b_n_of_matches, b_corners_, b_n_of_matches, b_corners_ / b_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'Corners Against', b_n_of_matches, b_opp_corners_, opp_b_n_of_matches, b_opp_corners_ / opp_b_n_of_matches])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'Big Chances', b_n_of_matches, b_big_chances, b_n_of_matches, b_big_chances / b_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'Big Chances Against', b_n_of_matches, b_opp_big_chances, opp_b_n_of_matches, b_opp_big_chances / opp_b_n_of_matches])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'xG', b_n_of_matches, b_xg_created, b_n_of_matches, b_xg_created / b_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'xG Against', b_n_of_matches, b_opp_xg_created, opp_b_n_of_matches, b_opp_xg_created / opp_b_n_of_matches])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'xG Open Play', b_n_of_matches, b_open_play_xg_created, b_n_of_matches, b_open_play_xg_created / b_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'xG Open Play Against', b_n_of_matches, b_opp_open_play_xg_created, opp_b_n_of_matches, b_opp_open_play_xg_created / opp_b_n_of_matches])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'xG Set Play', b_n_of_matches, b_set_play_xg_created, b_n_of_matches, b_set_play_xg_created / b_n_of_matches])
    metrics_out.append([add_team_name, against_tier_, add_team_name +' Key Tier Comparison', 'xG Set Play Against', b_n_of_matches, b_opp_set_play_xg_created, opp_b_n_of_matches, b_opp_set_play_xg_created / opp_b_n_of_matches])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Key Tier Comparison', 'Possession %', b_n_of_matches, b_passes_, b_opp_passes_, b_possess])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Key Tier Comparison', 'Possession % Against', b_n_of_matches, b_opp_passes_, b_passes_, 100-b_possess])
    
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Tier Comparison', 'PPDA', b_n_of_matches,b_opp_own_half_passes, b_possessions_won_in_opp_half, b_offensive_recovery_intensity])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Tier Comparison', 'PPDA Against', b_n_of_matches,b_own_half_passes, b_opp_possessions_won_in_opp_half, b_opp_offensive_recovery_intensity])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Tier Comparison', 'Efficacy High Block', b_n_of_matches,b_final_third_possessions_won, b_time_high_block, b_efficacy_high_block])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Tier Comparison', 'Efficacy Mid Block', b_n_of_matches,b_mid_third_possessions_won, b_time_mid_block, b_efficacy_mid_block])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Tier Comparison', 'Opponent Efficacy High Block', b_n_of_matches,b_opp_final_third_possessions_won, b_time_build_phase, b_opp_efficacy_high_block])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Tier Comparison', 'Opponent Efficacy Mid Block', b_n_of_matches,b_opp_mid_third_possessions_won, b_time_mid_phase, b_opp_efficacy_mid_block])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Tier Comparison', 'Offensive Efficacy', b_n_of_matches,b_open_play_xg_created, b_time_finish_phase, b_offensive_efficacy])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Tier Comparison', 'Defensive Efficacy', b_n_of_matches,b_opp_open_play_xg_created, b_time_low_block, b_defensive_efficacy])
    metrics_out.append([add_team_name, against_tier_, add_team_name+' Advanced Tier Comparison', 'Field Tilt', b_n_of_matches,b_time_finish_phase, b_time_low_block, b_field_tilt])


Tier: 0
 
Possession (%): 
76009.0 58506.0 56.50596587741145
PPDA: 
34333.0 3422.0 10.03302162478083
PPDA Against: 
39933.0 2811.0 14.205976520811099
Efficacy High Block: 
1281.0 54293.556 7.078188063423218
Efficacy Mid Block: 
4921.0 76877.44 19.20329292962929
Opponent Efficacy in High Block: 
1004.0 62699.795 4.8038434575424045
Opponent Efficacy in Mid Block: 
4350.0 111536.314 11.700225273716683
Offensive Efficacy: 
218.79056799039245 75936.908 0.8643645379545575
Defensive Efficacy: 
139.1300078202039 47807.104999999996 0.8730711124645839
Field Tilt: 
75936.908 47807.104999999996 61.366126860618294
Tier: 1
 
Possession (%): 
26612.0 26612.0 50.0
PPDA: 
14765.0 1255.0 11.764940239043824
PPDA Against: 
14765.0 1255.0 11.764940239043824
Efficacy High Block: 
449.0 24289.180999999997 5.545678958874736
Efficacy Mid Block: 
1846.0 37636.342 14.714501212684274
Opponent Efficacy in High Block: 
449.0 24289.180999999997 5.545678958874736
Opponent Efficacy in Mid Block: 
1846.0 37636.342 14.7

In [570]:
benchmark_teamed_melted_kpis_pre_f[benchmark_teamed_melted_kpis_pre_f['KPI'] == 'Opta_Passes_in_Own_Half']

,team_tier,KPI,Value
3,1,Opta_Passes_in_Own_Half,14045.0


In [571]:
aaa = first_tier_benchmark_metrics[first_tier_benchmark_metrics['against_tier'] == 1].groupby(by = ['game_id','Team','against_tier'])['xG'].sum().reset_index()
aaa['xG'].mean()

np.float64(1.4554433794307216)

In [572]:
advanced_metrics_out = pd.DataFrame(metrics_out, columns = ['team_name','against_tier','aggregation_type','KPI','n_of_matches','Val 1','Val 2','Value'])

display_all(advanced_metrics_out[advanced_metrics_out['KPI'] == 'Possession %'])

,team_name,against_tier,aggregation_type,KPI,n_of_matches,Val 1,Val 2,Value
12,England U17,0,England U17 Key Current,Possession %,5,1975.0,1692.0,53.858740
35,England U17,0,England U17 Key Benchmark,Possession %,59,30459.0,23132.0,56.836036
58,England U17,1,England U17 Key Benchmark,Possession %,20,9692.0,9010.0,51.823334
81,England U17,2,England U17 Key Benchmark,Possession %,22,12256.0,8073.0,60.288258
104,England U17,3,England U17 Key Benchmark,Possession %,17,8511.0,6049.0,58.454670
127,England U17,0,England U17 Key Tier Comparison,Possession %,155,76009.0,58506.0,56.505966
150,England U17,1,England U17 Key Tier Comparison,Possession %,60,26612.0,26612.0,50.000000
173,England U17,2,England U17 Key Tier Comparison,Possession %,39,21056.0,14228.0,59.675774
196,England U17,3,England U17 Key Tier Comparison,Possession %,56,28341.0,17666.0,61.601495


In [573]:
tier_comparison = advanced_metrics_out[advanced_metrics_out['aggregation_type'].str.contains(' Tier Comparison')]


In [574]:
benchmark_comparison = advanced_metrics_out[advanced_metrics_out['aggregation_type'].str.contains(' Benchmark')]

display_all(benchmark_comparison[benchmark_comparison['KPI'] == 'xG'])

,team_name,against_tier,aggregation_type,KPI,n_of_matches,Val 1,Val 2,Value
29,England U17,0,England U17 Key Benchmark,xG,59,99.038140,59.0,1.678613
52,England U17,1,England U17 Key Benchmark,xG,20,26.181938,20.0,1.309097
75,England U17,2,England U17 Key Benchmark,xG,22,29.953165,22.0,1.361507
98,England U17,3,England U17 Key Benchmark,xG,17,42.903037,17.0,2.523708


In [575]:

advanced_metrics_out_pre_j = pd.merge(advanced_metrics_out, tier_comparison.drop(columns = {'Val 1','Val 2','aggregation_type','n_of_matches'}).rename(columns = {'Value':'Comp_Value'}), on = ['team_name','KPI','against_tier'], how = 'left')
advanced_metrics_out_j = pd.merge(advanced_metrics_out_pre_j, benchmark_comparison.drop(columns = {'Val 1','Val 2','aggregation_type','n_of_matches'}).rename(columns = {'Value':'Benchmark_Value'}), on = ['team_name','KPI','against_tier'], how = 'left')

advanced_metrics_out_j_positive = advanced_metrics_out_j[advanced_metrics_out_j['KPI'].isin(['PPDA Against','Efficacy High Block','Efficacy Mid Block','Offensive Efficacy','Field Tilt','Possession %','Shots','Corners','Big Chances','xG','xG Open Play','xG Set Play'])]
advanced_metrics_out_j_negative = advanced_metrics_out_j[advanced_metrics_out_j['KPI'].isin(['PPDA','Opponent Efficacy High Block','Opponent Efficacy Mid Block','Defensive Efficacy','Possession % Against','Shots Against','Corners Against','Big Chances Against','xG Against','xG Open Play Against','xG Set Play Against'])]

advanced_metrics_out_j_positive['Value_norm_to_tier'] = (advanced_metrics_out_j_positive['Value'] - advanced_metrics_out_j_positive['Comp_Value'])/ advanced_metrics_out_j_positive['Comp_Value']
advanced_metrics_out_j_negative['Value_norm_to_tier'] = (advanced_metrics_out_j_negative['Comp_Value'] - advanced_metrics_out_j_negative['Value'])/ advanced_metrics_out_j_negative['Comp_Value']

advanced_metrics_out_j_positive['Value_norm_to_eng_benchmark'] = (advanced_metrics_out_j_positive['Value'] - advanced_metrics_out_j_positive['Benchmark_Value'])/ advanced_metrics_out_j_positive['Benchmark_Value']
advanced_metrics_out_j_negative['Value_norm_to_eng_benchmark'] = (advanced_metrics_out_j_negative['Benchmark_Value'] - advanced_metrics_out_j_negative['Value'])/ advanced_metrics_out_j_negative['Benchmark_Value']

advanced_metrics_out_jjj = pd.concat([advanced_metrics_out_j_positive, advanced_metrics_out_j_negative], ignore_index = True)

advanced_metrics_out_jjj['Opposition'] = 'Aggregate'
advanced_metrics_out_jjj['game_id'] = 99999

display_all(advanced_metrics_out_jjj[advanced_metrics_out_jjj['KPI'] == 'xG'])

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/2955047635.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  advanced_metrics_out_j_positive['Value_norm_to_tier'] = (advanced_metrics_out_j_positive['Value'] - advanced_metrics_out_j_positive['Comp_Value'])/ advanced_metrics_out_j_positive['Comp_Value']
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/2955047635.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  advanced_metrics_out_j_negative['Value_norm_to_tier'] = (advance

,team_name,against_tier,aggregation_type,KPI,n_of_matches,Val 1,Val 2,Value,Comp_Value,Benchmark_Value,Value_norm_to_tier,Value_norm_to_eng_benchmark,Opposition,game_id
3,England U17,0,England U17 Key Current,xG,5,12.138418,5.0,2.427684,1.879557,1.678613,0.291625,0.446244,Aggregate,99999
15,England U17,0,England U17 Key Benchmark,xG,59,99.038140,59.0,1.678613,1.879557,1.678613,-0.106911,0.000000,Aggregate,99999
27,England U17,1,England U17 Key Benchmark,xG,20,26.181938,20.0,1.309097,1.455443,1.309097,-0.100551,0.000000,Aggregate,99999
39,England U17,2,England U17 Key Benchmark,xG,22,29.953165,22.0,1.361507,1.638822,1.361507,-0.169216,0.000000,Aggregate,99999
51,England U17,3,England U17 Key Benchmark,xG,17,42.903037,17.0,2.523708,2.501619,2.523708,0.008830,0.000000,Aggregate,99999
63,England U17,0,England U17 Key Tier Comparison,xG,155,291.331352,155.0,1.879557,1.879557,1.678613,0.000000,0.119709,Aggregate,99999
75,England U17,1,England U17 Key Tier Comparison,xG,60,87.326603,60.0,1.455443,1.455443,1.309097,0.000000,0.111792,Aggregate,99999
87,England U17,2,England U17 Key Tier Comparison,xG,39,63.914064,39.0,1.638822,1.638822,1.361507,0.000000,0.203682,Aggregate,99999
99,England U17,3,England U17 Key Tier Comparison,xG,56,140.090685,56.0,2.501619,2.501619,2.523708,0.000000,-0.008752,Aggregate,99999


In [576]:
keep_vars = ['Shots','Big_Chances','Corners','xG','Open_Play_xG','Passes']

game_by_game_metrics = teamed_melted_kpis_pre_2[teamed_melted_kpis_pre_2['KPI'].isin(keep_vars)].groupby(by = ['Team','Opposition','against_tier','game_id','KPI'])['Value'].sum().reset_index()
opp_game_by_game_metrics = opp_teamed_melted_kpis_pre[opp_teamed_melted_kpis_pre['KPI'].isin(keep_vars)].groupby(by = ['Team','Opposition','team_tier','game_id','KPI'])['Value'].sum().reset_index()

opp_game_by_game_metrics[opp_game_by_game_metrics['game_id'] == 2507727]

,Team,Opposition,team_tier,game_id,KPI,Value


In [577]:
game_by_game_metrics[game_by_game_metrics['game_id'] == 2507727]

,Team,Opposition,against_tier,game_id,KPI,Value


In [578]:
add_metrics_data_list = []

for game_ in game_by_game_metrics.game_id.unique():
    
    team_ = game_by_game_metrics[(game_by_game_metrics['game_id'] == game_)]['Team'].iloc[0]
    opposition_ = game_by_game_metrics[(game_by_game_metrics['game_id'] == game_)]['Opposition'].iloc[0]
    against_tier = game_by_game_metrics[(game_by_game_metrics['game_id'] == game_)]['against_tier'].iloc[0]
    
    passes_ = game_by_game_metrics[(game_by_game_metrics['game_id'] == game_) & (game_by_game_metrics['KPI'] == 'Passes')]['Value'].sum()
    opp_passes_ = opp_game_by_game_metrics[(opp_game_by_game_metrics['game_id'] == game_) & (opp_game_by_game_metrics['KPI'] == 'Passes')]['Value'].sum()
    possess = 100*(passes_ / (passes_ + opp_passes_))
    opp_possess = 100 - possess
    xg_ = game_by_game_metrics[(game_by_game_metrics['game_id'] == game_) & (game_by_game_metrics['KPI'] == 'xG')]['Value'].sum()
    xg_open_play = game_by_game_metrics[(game_by_game_metrics['game_id'] == game_) & (game_by_game_metrics['KPI'] == 'Open_Play_xG')]['Value'].sum()
    xg_set_play = xg_ - xg_open_play
    opp_xg_ = opp_game_by_game_metrics[(opp_game_by_game_metrics['game_id'] == game_) & (opp_game_by_game_metrics['KPI'] == 'xG')]['Value'].sum()
    opp_xg_open_play = opp_game_by_game_metrics[(opp_game_by_game_metrics['game_id'] == game_) & (opp_game_by_game_metrics['KPI'] == 'Open_Play_xG')]['Value'].sum()
    opp_xg_set_play = opp_xg_ - opp_xg_open_play

    add_metrics_data_list.append([team_, opposition_, against_tier, game_, 'Possession %', possess])
    add_metrics_data_list.append([team_, opposition_, against_tier, game_, 'Possession % Against', opp_possess])
    add_metrics_data_list.append([team_, opposition_, against_tier, game_, 'xG Set Play', xg_set_play])
    add_metrics_data_list.append([team_, opposition_, against_tier, game_, 'xG Set Play Against', opp_xg_set_play])
    
add_metrics_data = pd.DataFrame(add_metrics_data_list, columns = ['team_name','Opposition','against_tier','game_id','KPI','Value'])

game_by_game_metrics_r = game_by_game_metrics.rename(columns = {'Team':'team_name'})
game_by_game_metrics_r['KPI'] = game_by_game_metrics_r['KPI'].str.replace('_', ' ').str.replace('Open Play xG', 'xG Open Play')

opp_game_by_game_metrics_r1 = opp_game_by_game_metrics.rename(columns = {'Opposition':'team_name','team_tier':'against_tier'})
opp_game_by_game_metrics_r2 = opp_game_by_game_metrics_r1.rename(columns = {'Team':'Opposition'})
opp_game_by_game_metrics_r = opp_game_by_game_metrics_r2[game_by_game_metrics_r.columns]

opp_game_by_game_metrics_r['KPI'] = opp_game_by_game_metrics_r['KPI'].str.replace('_', ' ').str.replace('Open Play xG', 'xG Open Play')
opp_game_by_game_metrics_r['KPI'] = opp_game_by_game_metrics_r['KPI'] + ' Against'

game_by_game_metrics_r_merged = pd.concat([game_by_game_metrics_r, opp_game_by_game_metrics_r, add_metrics_data], ignore_index = True)

game_by_game_metrics_pre_j = pd.merge(game_by_game_metrics_r_merged, tier_comparison.drop(columns = {'Val 1','Val 2','aggregation_type','n_of_matches'}).rename(columns = {'Value':'Comp_Value'}), on = ['team_name','KPI','against_tier'], how = 'left')
game_by_game_metrics_j = pd.merge(game_by_game_metrics_pre_j, benchmark_comparison.drop(columns = {'Val 1','Val 2','aggregation_type','n_of_matches'}).rename(columns = {'Value':'Benchmark_Value'}), on = ['team_name','KPI','against_tier'], how = 'left')

game_by_game_metrics_j_filt = game_by_game_metrics_j[~game_by_game_metrics_j['KPI'].str.contains('Passes')]

#FINISH UP THIS!
game_by_game_metrics_j_filt_positive = game_by_game_metrics_j_filt[game_by_game_metrics_j_filt['KPI'].isin(['Possession %','Shots','Corners','Big Chances','xG','xG Open Play','xG Set Play'])]
game_by_game_metrics_j_filt_negative = game_by_game_metrics_j_filt[game_by_game_metrics_j_filt['KPI'].isin(['Possession % Against','Shots Against','Corners Against','Big Chances Against','xG Against','xG Open Play Against','xG Set Play Against'])]

game_by_game_metrics_j_filt_positive['Value_norm_to_tier'] = (game_by_game_metrics_j_filt_positive['Value'] - game_by_game_metrics_j_filt_positive['Comp_Value'])/ game_by_game_metrics_j_filt_positive['Comp_Value']
game_by_game_metrics_j_filt_negative['Value_norm_to_tier'] = (game_by_game_metrics_j_filt_negative['Comp_Value'] - game_by_game_metrics_j_filt_negative['Value'])/ game_by_game_metrics_j_filt_negative['Comp_Value']

game_by_game_metrics_j_filt_positive['Value_norm_to_eng_benchmark'] = (game_by_game_metrics_j_filt_positive['Value'] - game_by_game_metrics_j_filt_positive['Benchmark_Value'])/ game_by_game_metrics_j_filt_positive['Benchmark_Value']
game_by_game_metrics_j_filt_negative['Value_norm_to_eng_benchmark'] = (game_by_game_metrics_j_filt_negative['Benchmark_Value'] - game_by_game_metrics_j_filt_negative['Value'])/ game_by_game_metrics_j_filt_negative['Benchmark_Value']

game_by_game_metrics_j_out = pd.concat([game_by_game_metrics_j_filt_positive, game_by_game_metrics_j_filt_negative], ignore_index = True)

game_by_game_metrics_j_out_fff = pd.merge(game_by_game_metrics_j_out, add_date, on = ['game_id'], how='left')
game_by_game_metrics_j_out_fff['aggregation_type'] = add_team_name + ' Single Game Metrics'
game_by_game_metrics_j_out_fff['Val 1'] = 0.0
game_by_game_metrics_j_out_fff['Val 2'] = 0.0
game_by_game_metrics_j_out_fff['n_of_matches'] = 1.0

display_all(game_by_game_metrics_j_out_fff.sort_values(by = ['game_id','against_tier']))

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3797548116.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_by_game_metrics_j_filt_positive['Value_norm_to_tier'] = (game_by_game_metrics_j_filt_positive['Value'] - game_by_game_metrics_j_filt_positive['Comp_Value'])/ game_by_game_metrics_j_filt_positive['Comp_Value']
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3797548116.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_by_game_metrics_j_filt_negative['Valu

,team_name,Opposition,against_tier,game_id,KPI,Value,Comp_Value,Benchmark_Value,Value_norm_to_tier,Value_norm_to_eng_benchmark,matchDateTime,aggregation_type,Val 1,Val 2,n_of_matches
20,England U17,Venezuela U17,3,2556968,Big Chances,2.0,3.571429,3.588235,-0.44,-0.442623,2025-11-04,England U17 Single Game Metrics,0.0,0.0,1.0
21,England U17,Venezuela U17,3,2556968,Corners,7.0,6.303571,5.235294,0.110482,0.337079,2025-11-04,England U17 Single Game Metrics,0.0,0.0,1.0
22,England U17,Venezuela U17,3,2556968,xG Open Play,1.513324,1.837713,2.001891,-0.176518,-0.244053,2025-11-04,England U17 Single Game Metrics,0.0,0.0,1.0
23,England U17,Venezuela U17,3,2556968,Shots,18.0,18.928571,17.352941,-0.049057,0.037288,2025-11-04,England U17 Single Game Metrics,0.0,0.0,1.0
24,England U17,Venezuela U17,3,2556968,xG,1.981214,2.501619,2.523708,-0.208027,-0.214959,2025-11-04,England U17 Single Game Metrics,0.0,0.0,1.0
33,England U17,Venezuela U17,3,2556968,Possession %,56.414219,61.601495,58.454670,-0.084207,-0.034907,2025-11-04,England U17 Single Game Metrics,0.0,0.0,1.0
34,England U17,Venezuela U17,3,2556968,xG Set Play,0.46789,0.663907,0.521817,-0.295247,-0.103345,2025-11-04,England U17 Single Game Metrics,0.0,0.0,1.0
55,England U17,Venezuela U17,3,2556968,Big Chances Against,1.0,1.071429,1.000000,0.066667,0.0,2025-11-04,England U17 Single Game Metrics,0.0,0.0,1.0
56,England U17,Venezuela U17,3,2556968,Corners Against,4.0,3.375000,2.882353,-0.185185,-0.387755,2025-11-04,England U17 Single Game Metrics,0.0,0.0,1.0
57,England U17,Venezuela U17,3,2556968,xG Open Play Against,1.019662,0.636540,0.715725,-0.601884,-0.424656,2025-11-04,England U17 Single Game Metrics,0.0,0.0,1.0


In [579]:
from datetime import date 

today = date.today()
today

datetime.date(2026, 1, 29)

In [580]:
advanced_metrics_out_jjj['matchDateTime'] = today

game_by_game_metrics_j_out_fff_s = game_by_game_metrics_j_out_fff[advanced_metrics_out_jjj.columns]

total_metrics_out = pd.concat([advanced_metrics_out_jjj, game_by_game_metrics_j_out_fff_s], ignore_index = True)

display_all(total_metrics_out.sort_values(by  = ['matchDateTime']))

,team_name,against_tier,aggregation_type,KPI,n_of_matches,Val 1,Val 2,Value,Comp_Value,Benchmark_Value,Value_norm_to_tier,Value_norm_to_eng_benchmark,Opposition,game_id,matchDateTime
276,England U17,3,England U17 Single Game Metrics,xG Set Play Against,1.0,0.000000,0.000,0.922223,0.299575,0.281541,-2.07844,-2.275631,Venezuela U17,2556968,2025-11-04
227,England U17,3,England U17 Single Game Metrics,Big Chances,1.0,0.000000,0.000,2.0,3.571429,3.588235,-0.44,-0.442623,Venezuela U17,2556968,2025-11-04
228,England U17,3,England U17 Single Game Metrics,Corners,1.0,0.000000,0.000,7.0,6.303571,5.235294,0.110482,0.337079,Venezuela U17,2556968,2025-11-04
229,England U17,3,England U17 Single Game Metrics,xG Open Play,1.0,0.000000,0.000,1.513324,1.837713,2.001891,-0.176518,-0.244053,Venezuela U17,2556968,2025-11-04
230,England U17,3,England U17 Single Game Metrics,Shots,1.0,0.000000,0.000,18.0,18.928571,17.352941,-0.049057,0.037288,Venezuela U17,2556968,2025-11-04
231,England U17,3,England U17 Single Game Metrics,xG,1.0,0.000000,0.000,1.981214,2.501619,2.523708,-0.208027,-0.214959,Venezuela U17,2556968,2025-11-04
240,England U17,3,England U17 Single Game Metrics,Possession %,1.0,0.000000,0.000,56.414219,61.601495,58.454670,-0.084207,-0.034907,Venezuela U17,2556968,2025-11-04
241,England U17,3,England U17 Single Game Metrics,xG Set Play,1.0,0.000000,0.000,0.46789,0.663907,0.521817,-0.295247,-0.103345,Venezuela U17,2556968,2025-11-04
262,England U17,3,England U17 Single Game Metrics,Big Chances Against,1.0,0.000000,0.000,1.0,1.071429,1.000000,0.066667,0.0,Venezuela U17,2556968,2025-11-04
263,England U17,3,England U17 Single Game Metrics,Corners Against,1.0,0.000000,0.000,4.0,3.375000,2.882353,-0.185185,-0.387755,Venezuela U17,2556968,2025-11-04


In [581]:
thresholds = tier_team_metrics[['team_tier','against_tier','KPI','Value']].drop_duplicates()
thresholds[thresholds['team_tier'] == 1]

,team_tier,against_tier,KPI,Value
18,1.0,1.0,% IN BUILD PHASE,14.18
19,1.0,1.0,% IN CREATE PHASE,21.97
20,1.0,1.0,% IN FINISH PHASE,13.86
21,1.0,2.0,% IN BUILD PHASE,15.93
22,1.0,2.0,% IN CREATE PHASE,28.27
23,1.0,2.0,% IN FINISH PHASE,17.63
24,1.0,3.0,% IN BUILD PHASE,14.15
25,1.0,3.0,% IN CREATE PHASE,28.95
26,1.0,3.0,% IN FINISH PHASE,22.14
27,1.0,1.0,% IN HIGH BLOCK,14.18


In [582]:
eng_benchmark_thresholds = eng_tier_team_metrics[['team_tier','against_tier','KPI','Value']].drop_duplicates()
eng_benchmark_thresholds

,team_tier,against_tier,KPI,Value
0,1.0,1.0,% IN BUILD PHASE,14.05
1,1.0,1.0,% IN CREATE PHASE,24.06
2,1.0,1.0,% IN FINISH PHASE,15.37
3,1.0,2.0,% IN BUILD PHASE,16.97
4,1.0,2.0,% IN CREATE PHASE,27.69
5,1.0,2.0,% IN FINISH PHASE,19.20
6,1.0,3.0,% IN BUILD PHASE,15.17
7,1.0,3.0,% IN CREATE PHASE,28.43
8,1.0,3.0,% IN FINISH PHASE,19.48
9,1.0,1.0,% IN HIGH BLOCK,16.48


In [583]:
phases_by_game_query = """
SELECT *
FROM `the-fa-performance-insights.dev_vanni.phases_metrics_by_game`
"""

phases_by_game = benchmark_games[benchmark_games['game_id'].isin(game_by_game_metrics_j_out['game_id'].unique())]

phases_by_game_add_1 = pd.merge(phases_by_game, thresholds.rename(columns = {'Value':'Comp_Value'}), on = ['team_tier','against_tier','KPI'], how = 'left')
phases_by_game_add = pd.merge(phases_by_game_add_1, eng_benchmark_thresholds.rename(columns = {'Value':'Benchmark_Value'}), on = ['team_tier','against_tier','KPI'], how = 'left')

phases_by_game_add.sort_values(by = ['phase'])

,event_team_name,game_id,against_team_name,team_tier,against_tier,phase,time_diff,KPI,Value,n_of_matches,total_create_to_finish,successful_create_to_finish,Comp_Value,Benchmark_Value
18,England U17,2556970,Haiti U17,1.0,3.0,% IN HIGH BLOCK,262.408,% IN HIGH BLOCK,9.61,9,NaN,NaN,10.66,13.17
66,Austria U17,2609005,England U17,2.0,1.0,% IN HIGH BLOCK,274.635,% IN HIGH BLOCK,11.45,8,NaN,NaN,16.26,NaN
15,England U17,2556968,Venezuela U17,1.0,3.0,% IN HIGH BLOCK,268.505,% IN HIGH BLOCK,10.07,9,NaN,NaN,10.66,13.17
59,Haiti U17,2556970,England U17,3.0,1.0,% IN HIGH BLOCK,595.448,% IN HIGH BLOCK,21.81,3,NaN,NaN,14.15,NaN
21,England U17,2556972,Egypt U17,1.0,3.0,% IN HIGH BLOCK,292.214,% IN HIGH BLOCK,10.77,9,NaN,NaN,10.66,13.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,Korea Republic U17,2608983,None,NaN,NaN,None,NaN,Create to Finish Success Rate (%),13.21,7,106.0,14.0,NaN,NaN
48,Venezuela U17,2556968,None,NaN,NaN,None,NaN,Create to Finish Success Rate (%),12.66,8,79.0,10.0,NaN,NaN
55,Egypt U17,2556972,None,NaN,NaN,None,NaN,Create to Finish Success Rate (%),8.70,4,92.0,8.0,NaN,NaN
62,Haiti U17,2556970,None,NaN,NaN,None,NaN,Create to Finish Success Rate (%),14.63,3,82.0,12.0,NaN,NaN


In [584]:
#FINISH UP THIS!
phases_by_game_add_positive = phases_by_game_add[phases_by_game_add['KPI'].str.contains('PHASE')]
phases_by_game_add_negative = phases_by_game_add[phases_by_game_add['KPI'].str.contains('BLOCK')]

phases_by_game_add_positive['Value_norm_to_tier'] = (phases_by_game_add_positive['Value'] - phases_by_game_add_positive['Comp_Value'])/ phases_by_game_add_positive['Comp_Value']
phases_by_game_add_negative['Value_norm_to_tier'] = (phases_by_game_add_negative['Value'] - phases_by_game_add_negative['Comp_Value'])/ phases_by_game_add_negative['Comp_Value']

phases_by_game_add_positive['Value_norm_to_eng_benchmark'] = (phases_by_game_add_positive['Value'] - phases_by_game_add_positive['Benchmark_Value'])/ phases_by_game_add_positive['Benchmark_Value']
phases_by_game_add_negative['Value_norm_to_eng_benchmark'] = (phases_by_game_add_negative['Value'] - phases_by_game_add_negative['Benchmark_Value'])/ phases_by_game_add_negative['Benchmark_Value']

phases_by_game_out = pd.concat([phases_by_game_add_positive, phases_by_game_add_negative], ignore_index = True)
phases_by_game_out_f = phases_by_game_out[phases_by_game_out['event_team_name'] == add_team_name]

phases_by_game_out_fff = pd.merge(phases_by_game_out_f, add_date, on = ['game_id'], how='left')
phases_by_game_out_fff

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/1720866760.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phases_by_game_add_positive['Value_norm_to_tier'] = (phases_by_game_add_positive['Value'] - phases_by_game_add_positive['Comp_Value'])/ phases_by_game_add_positive['Comp_Value']
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/1720866760.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phases_by_game_add_negative['Value_norm_to_tier'] = (phases_by_game_add_negative

,event_team_name,game_id,against_team_name,team_tier,against_tier,phase,time_diff,KPI,Value,n_of_matches,total_create_to_finish,successful_create_to_finish,Comp_Value,Benchmark_Value,Value_norm_to_tier,Value_norm_to_eng_benchmark,matchDateTime
0,England U17,2556968,Venezuela U17,1.0,3.0,BUILD,394.222,% IN BUILD PHASE,14.79,9,NaN,NaN,14.15,15.17,0.045230,-0.025049,2025-11-04
1,England U17,2556968,Venezuela U17,1.0,3.0,CREATE,818.786,% IN CREATE PHASE,30.71,9,NaN,NaN,28.95,28.43,0.060794,0.080197,2025-11-04
2,England U17,2556968,Venezuela U17,1.0,3.0,FINISH,535.010,% IN FINISH PHASE,20.07,9,NaN,NaN,22.14,19.48,-0.093496,0.030287,2025-11-04
3,England U17,2556970,Haiti U17,1.0,3.0,BUILD,595.448,% IN BUILD PHASE,21.81,9,NaN,NaN,14.15,15.17,0.541343,0.437706,2025-11-07
4,England U17,2556970,Haiti U17,1.0,3.0,CREATE,659.235,% IN CREATE PHASE,24.14,9,NaN,NaN,28.95,28.43,-0.166149,-0.150897,2025-11-07
5,England U17,2556970,Haiti U17,1.0,3.0,FINISH,624.183,% IN FINISH PHASE,22.86,9,NaN,NaN,22.14,19.48,0.032520,0.173511,2025-11-07
6,England U17,2556972,Egypt U17,1.0,3.0,BUILD,357.738,% IN BUILD PHASE,13.18,9,NaN,NaN,14.15,15.17,-0.068551,-0.131180,2025-11-10
7,England U17,2556972,Egypt U17,1.0,3.0,CREATE,794.425,% IN CREATE PHASE,29.27,9,NaN,NaN,28.95,28.43,0.011054,0.029546,2025-11-10
8,England U17,2556972,Egypt U17,1.0,3.0,FINISH,323.801,% IN FINISH PHASE,11.93,9,NaN,NaN,22.14,19.48,-0.461156,-0.387577,2025-11-10
9,England U17,2608983,Korea Republic U17,1.0,3.0,BUILD,355.298,% IN BUILD PHASE,11.72,9,NaN,NaN,14.15,15.17,-0.171731,-0.227423,2025-11-15


In [585]:
aggr_phases_pre_norm = pd.merge(aggr_phases_metrics, first_tier_team_metrics_out.rename(columns = {'time_diff':'Comp_Time_Diff', 'Value':'Comp_Value','n_of_matches':'t1_matches'}), on = ['KPI'])
aggr_phases_norm = pd.merge(aggr_phases_pre_norm, eng_first_tier_team_metrics_out.rename(columns = {'time_diff':'Benchmark_Time_Diff', 'Value':'Benchmark_Value','n_of_matches':'eng_matches'}), on = ['KPI'])

aggr_phases_norm_positive = aggr_phases_norm[aggr_phases_norm['KPI'].str.contains('PHASE')]
aggr_phases_norm_negative = aggr_phases_norm[aggr_phases_norm['KPI'].str.contains('BLOCK')]

aggr_phases_norm_positive['Value_norm_to_tier'] = (aggr_phases_norm_positive['Value'] - aggr_phases_norm_positive['Comp_Value'])/ aggr_phases_norm_positive['Comp_Value']
aggr_phases_norm_negative['Value_norm_to_tier'] = (aggr_phases_norm_negative['Value'] - aggr_phases_norm_negative['Comp_Value'])/ aggr_phases_norm_negative['Comp_Value']

aggr_phases_norm_positive['Value_norm_to_eng_benchmark'] = (aggr_phases_norm_positive['Value'] - aggr_phases_norm_positive['Benchmark_Value'])/ aggr_phases_norm_positive['Benchmark_Value']
aggr_phases_norm_negative['Value_norm_to_eng_benchmark'] = (aggr_phases_norm_negative['Value'] - aggr_phases_norm_negative['Benchmark_Value'])/ aggr_phases_norm_negative['Benchmark_Value']

aggr_phases_norm_out = pd.concat([aggr_phases_norm_positive, aggr_phases_norm_negative], ignore_index = True)

aggr_phases_norm_out['minutes_spent_floating'] = aggr_phases_norm_out['time_diff'] / (60*aggr_phases_norm_out['n_of_matches'])
aggr_phases_norm_out['t1_benchmark_minutes_spent_floating'] = aggr_phases_norm_out['Comp_Time_Diff'] / (60*aggr_phases_norm_out['t1_matches'])
aggr_phases_norm_out['eng_benchmark_minutes_spent_floating'] = aggr_phases_norm_out['Benchmark_Time_Diff'] / (60*aggr_phases_norm_out['eng_matches'])

aggr_phases_norm_out

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/2221550174.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggr_phases_norm_positive['Value_norm_to_tier'] = (aggr_phases_norm_positive['Value'] - aggr_phases_norm_positive['Comp_Value'])/ aggr_phases_norm_positive['Comp_Value']
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/2221550174.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggr_phases_norm_negative['Value_norm_to_tier'] = (aggr_phases_norm_negative['Value'] - 

,event_team_name,KPI,time_diff,Value,n_of_matches,Comp_Time_Diff,Comp_Value,t1_matches,Benchmark_Time_Diff,Benchmark_Value,eng_matches,Value_norm_to_tier,Value_norm_to_eng_benchmark,minutes_spent_floating,t1_benchmark_minutes_spent_floating,eng_benchmark_minutes_spent_floating
0,England U17,% IN BUILD PHASE,1983.164,15.663419,5,62699.795,14.61,153,26063.709,15.43,57,0.072103,0.015128,6.610547,6.830043,7.620968
1,England U17,% IN CREATE PHASE,3218.633,25.421396,5,111536.314,25.99,153,44941.788,26.61,57,-0.021878,-0.044668,10.728777,12.149925,13.140874
2,England U17,% IN FINISH PHASE,2124.512,16.779814,5,75936.908,17.69,153,30275.683,17.93,57,-0.051452,-0.064149,7.081707,8.271994,8.852539
3,England U17,% IN HIGH BLOCK,1695.088,13.388138,5,54293.556,12.65,153,24398.594,14.45,57,0.058351,-0.073485,5.650293,5.914331,7.134092
4,England U17,% IN MID BLOCK,2018.865,15.945393,5,76877.440,17.91,153,26801.133,15.87,57,-0.109693,0.004751,6.72955,8.374449,7.836589
5,England U17,% IN LOW BLOCK,1620.856,12.801839,5,47807.105,11.14,153,16401.158,9.71,57,0.149178,0.318418,5.402853,5.207746,4.795660


In [586]:
replaceOrAppend

'append'

In [587]:
phases_by_game_out_fff.to_gbq(
            destination_table='dev_vanni.phases_metrics_by_game_upd',
            project_id='the-fa-performance-insights',
            if_exists=replaceOrAppend
        )

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3345224468.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  phases_by_game_out_fff.to_gbq(
100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 10082.46it/s]


In [588]:
aggr_phases_norm_out.to_gbq(
            destination_table='dev_vanni.phases_metrics_aggregate_upd',
            project_id='the-fa-performance-insights',
            if_exists=replaceOrAppend
        )

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/1335749697.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  aggr_phases_norm_out.to_gbq(
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 9892.23it/s]


In [589]:
total_metrics_out.to_gbq(
            destination_table='dev_vanni.advanced_metrics_out',
            project_id='the-fa-performance-insights',
            if_exists=replaceOrAppend
        )

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/458564143.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  total_metrics_out.to_gbq(
100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 10866.07it/s]


In [590]:
momentum_metrics_out_jjj.to_gbq(
            destination_table='dev_vanni.momentum_metrics_out',
            project_id='the-fa-performance-insights',
            if_exists=replaceOrAppend
        )

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/1599037418.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  momentum_metrics_out_jjj.to_gbq(
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 5433.04it/s]


In [591]:
agg_momentum_metrics_out_jjj_fff.to_gbq(
            destination_table='dev_vanni.momentum_metrics_by_game',
            project_id='the-fa-performance-insights',
            if_exists=replaceOrAppend
        )

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/3685733479.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  agg_momentum_metrics_out_jjj_fff.to_gbq(
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 3310.42it/s]


In [592]:
phases_by_tier_metrics.to_gbq(
            destination_table='dev_vanni.phases_by_tier_metrics',
            project_id='the-fa-performance-insights',
            if_exists=replaceOrAppend
        )


/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_1840/1712729149.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  phases_by_tier_metrics.to_gbq(
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 4253.86it/s]


In [593]:
### testing results

In [594]:
sql_test_phases_metrics_by_game_upd = 'SELECT * FROM `the-fa-performance-insights.dev_vanni.phases_metrics_by_game_upd`'
test_phases_metrics_by_game_upd = pandas_gbq.read_gbq(sql_test_phases_metrics_by_game_upd, project_id = 'the-fa-performance-insights')

display_all(test_phases_metrics_by_game_upd[(test_phases_metrics_by_game_upd['phase'] == 'FINISH') & (test_phases_metrics_by_game_upd['team_tier'] == 1) & (test_phases_metrics_by_game_upd['against_tier'] == 3)].head(50))

Downloading: 100%|█████████████████████████████████████████████████████████████|


,event_team_name,game_id,against_team_name,team_tier,against_tier,phase,time_diff,KPI,Value,n_of_matches,total_create_to_finish,successful_create_to_finish,Comp_Value,Benchmark_Value,Value_norm_to_tier,Value_norm_to_eng_benchmark,matchDateTime
22,England U17,2556968,Venezuela U17,1.0,3.0,FINISH,535.010,% IN FINISH PHASE,20.07,9,NaN,NaN,22.14,19.48,-0.093496,0.030287,2025-11-04
23,England U17,2556970,Haiti U17,1.0,3.0,FINISH,624.183,% IN FINISH PHASE,22.86,9,NaN,NaN,22.14,19.48,0.032520,0.173511,2025-11-07
24,England U17,2556972,Egypt U17,1.0,3.0,FINISH,323.801,% IN FINISH PHASE,11.93,9,NaN,NaN,22.14,19.48,-0.461156,-0.387577,2025-11-10
25,England U17,2608983,Korea Republic U17,1.0,3.0,FINISH,481.092,% IN FINISH PHASE,15.87,9,NaN,NaN,22.14,19.48,-0.283198,-0.185318,2025-11-15
38,England U18,2602931,Uzbekistan U18,1.0,3.0,FINISH,356.516,% IN FINISH PHASE,15.69,48,NaN,NaN,22.14,19.48,-0.291328,-0.194559,2025-09-03
44,England U18,2602932,Canada U18,1.0,3.0,FINISH,306.793,% IN FINISH PHASE,11.82,48,NaN,NaN,22.14,19.48,-0.466125,-0.393224,2025-09-09
82,England U21,2507721,Kazakhstan U21,1.0,3.0,FINISH,487.859,% IN FINISH PHASE,14.77,51,NaN,NaN,25.46,25.77,-0.419874,-0.426853,2025-09-08
83,England U21,2507726,Moldova U21,1.0,3.0,FINISH,734.819,% IN FINISH PHASE,25.63,51,NaN,NaN,25.46,25.77,0.006677,-0.005433,2025-10-10
84,England U21,2507727,Andorra U21,1.0,3.0,FINISH,1368.497,% IN FINISH PHASE,37.28,51,NaN,NaN,25.46,25.77,0.464258,0.446643,2025-10-13
85,England U21,2507730,Republic of Ireland U21,1.0,3.0,FINISH,437.900,% IN FINISH PHASE,18.11,51,NaN,NaN,25.46,25.77,-0.288688,-0.297245,2025-11-14


In [595]:
sql_test_phases_metrics_aggregate_upd = 'SELECT * FROM `the-fa-performance-insights.dev_vanni.phases_metrics_aggregate_upd`'
test_phases_metrics_aggregate_upd = pandas_gbq.read_gbq(sql_test_phases_metrics_aggregate_upd, project_id = 'the-fa-performance-insights')

display_all(test_phases_metrics_aggregate_upd[test_phases_metrics_aggregate_upd['event_team_name'] == 'England U18'])

Downloading: 100%|█████████████████████████████████████████████████████████████|


,event_team_name,KPI,time_diff,Value,n_of_matches,Comp_Time_Diff,Comp_Value,t1_matches,Benchmark_Time_Diff,Benchmark_Value,eng_matches,Value_norm_to_tier,Value_norm_to_eng_benchmark,minutes_spent_floating,t1_benchmark_minutes_spent_floating,eng_benchmark_minutes_spent_floating
6,England U18,% IN BUILD PHASE,2214.943,15.852296,5,62699.795,14.61,153,26063.709,15.43,57,0.085031,0.027368,7.383143,6.830043,7.620968
7,England U18,% IN CREATE PHASE,4465.752,31.961284,5,111536.314,25.99,153,44941.788,26.61,57,0.229753,0.201100,14.885840,12.149925,13.140874
8,England U18,% IN FINISH PHASE,2019.335,14.452334,5,75936.908,17.69,153,30275.683,17.93,57,-0.183022,-0.193958,6.731117,8.271994,8.852539
9,England U18,% IN HIGH BLOCK,2335.927,16.718175,5,54293.556,12.65,153,24398.594,14.45,57,0.321595,0.156967,7.786423,5.914331,7.134092
10,England U18,% IN MID BLOCK,1856.143,13.284372,5,76877.440,17.91,153,26801.133,15.87,57,-0.258271,-0.162925,6.187143,8.374449,7.836589
11,England U18,% IN LOW BLOCK,1080.280,7.731539,5,47807.105,11.14,153,16401.158,9.71,57,-0.305966,-0.203755,3.600933,5.207746,4.795660


In [596]:
sql_test_advanced_metrics_input = 'SELECT * FROM `the-fa-performance-insights.dev_vanni.advanced_metrics_out`'
test_advanced_metrics_input = pandas_gbq.read_gbq(sql_test_advanced_metrics_input, project_id = 'the-fa-performance-insights')

display_all(test_advanced_metrics_input[(test_advanced_metrics_input['team_name'] == 'England U21') & (test_advanced_metrics_input['KPI'] == 'xG')].head(50))

Downloading: 100%|█████████████████████████████████████████████████████████████|


,team_name,against_tier,aggregation_type,KPI,n_of_matches,Val 1,Val 2,Value,Comp_Value,Benchmark_Value,Value_norm_to_tier,Value_norm_to_eng_benchmark,Opposition,game_id,matchDateTime
557,England U21,0,England U21 Key Current,xG,5.0,12.205950,5.0,2.441190,2.022836,2.151357,0.206816,0.134721,Aggregate,99999,2026-01-29
569,England U21,0,England U21 Key Benchmark,xG,51.0,109.719200,51.0,2.151357,2.022836,2.151357,0.063535,0.000000,Aggregate,99999,2026-01-29
581,England U21,1,England U21 Key Benchmark,xG,15.0,26.009921,15.0,1.733995,1.657371,1.733995,0.046232,0.000000,Aggregate,99999,2026-01-29
593,England U21,2,England U21 Key Benchmark,xG,14.0,31.967418,14.0,2.283387,2.148352,2.283387,0.062855,0.000000,Aggregate,99999,2026-01-29
605,England U21,3,England U21 Key Benchmark,xG,22.0,51.741860,22.0,2.351903,2.445782,2.351903,-0.038384,0.000000,Aggregate,99999,2026-01-29
617,England U21,0,England U21 Key Tier Comparison,xG,143.0,289.265553,143.0,2.022836,2.022836,2.151357,0.000000,-0.059739,Aggregate,99999,2026-01-29
629,England U21,1,England U21 Key Tier Comparison,xG,62.0,102.756980,62.0,1.657371,1.657371,1.733995,0.000000,-0.044189,Aggregate,99999,2026-01-29
641,England U21,2,England U21 Key Tier Comparison,xG,39.0,83.785727,39.0,2.148352,2.148352,2.283387,0.000000,-0.059138,Aggregate,99999,2026-01-29
653,England U21,3,England U21 Key Tier Comparison,xG,42.0,102.722846,42.0,2.445782,2.445782,2.351903,0.000000,0.039916,Aggregate,99999,2026-01-29
765,England U21,3,England U21 Single Game Metrics,xG,1.0,0.000000,0.0,3.519094,2.445782,2.351903,0.438842,0.496275,Andorra U21,2507727,2025-10-13


In [597]:
sql_test_momentum_metrics_out = 'SELECT * FROM `the-fa-performance-insights.dev_vanni.momentum_metrics_out`'
test_momentum_metrics_input = pandas_gbq.read_gbq(sql_test_momentum_metrics_out, project_id = 'the-fa-performance-insights')

display_all(test_momentum_metrics_input[(test_momentum_metrics_input['Team'] == 'England U21') & (test_momentum_metrics_input['against_tier'] == 0)].head(10))

Downloading: 100%|█████████████████████████████████████████████████████████████|


,team_tier,KPI,Comp_Value,against_tier,Team,Benchmark_Value,Value,Value_norm_to_tier,Value_norm_to_eng_benchmark,Opposition,game_id
20,1,Balanced,44.78,0,England U21,39.43,34.76,-0.223761,-0.118438,Aggregate,99999
21,1,Control,17.26,0,England U21,21.04,23.49,0.360950,0.116445,Aggregate,99999
22,1,High Control,21.47,0,England U21,29.50,39.78,0.852818,0.348475,Aggregate,99999
32,1,Opponent Control,8.74,0,England U21,4.51,1.97,0.774600,0.563193,Aggregate,99999
33,1,Opponent High Control,7.75,0,England U21,5.53,0.00,1.000000,1.000000,Aggregate,99999


In [598]:
sql_test_momentum_metrics_by_game = 'SELECT * FROM `the-fa-performance-insights.dev_vanni.momentum_metrics_by_game`'
test_momentum_metrics_by_game = pandas_gbq.read_gbq(sql_test_momentum_metrics_by_game, project_id = 'the-fa-performance-insights')

display_all(test_momentum_metrics_by_game[(test_momentum_metrics_by_game['Team'] == 'England U18')].head(10))

Downloading: 100%|█████████████████████████████████████████████████████████████|


,game_id,Team,Opposition,team_tier,against_tier,KPI,seq_count,total_seq_time_length,Value,Comp_Value,Benchmark_Value,Value_norm_to_tier,Value_norm_to_eng_benchmark,matchDateTime,game_description
0,2602848,England U18,Morocco U18,1,2,Balanced,71,1912.0,63.00,46.91,48.32,0.342997,0.303808,2025-09-06,England U18-Morocco U18
1,2602848,England U18,Morocco U18,1,2,Control,18,526.0,17.33,17.04,13.67,0.017019,0.267740,2025-09-06,England U18-Morocco U18
2,2602848,England U18,Morocco U18,1,2,High Control,34,597.0,19.67,16.45,13.43,0.195745,0.464631,2025-09-06,England U18-Morocco U18
3,2602931,England U18,Uzbekistan U18,1,3,Balanced,45,782.0,23.20,41.96,45.71,-0.447092,-0.492452,2025-09-03,England U18-Uzbekistan U18
4,2602931,England U18,Uzbekistan U18,1,3,Control,78,1518.0,45.04,16.94,18.76,1.658796,1.400853,2025-09-03,England U18-Uzbekistan U18
5,2602931,England U18,Uzbekistan U18,1,3,High Control,40,704.0,20.89,30.64,25.35,-0.318211,-0.175937,2025-09-03,England U18-Uzbekistan U18
6,2602931,England U18,Uzbekistan U18,1,3,Opponent Control,2,188.0,5.58,6.07,6.43,0.080725,0.132193,2025-09-03,England U18-Uzbekistan U18
7,2602931,England U18,Uzbekistan U18,1,3,Opponent High Control,6,178.0,5.28,4.39,3.76,-0.202733,-0.404255,2025-09-03,England U18-Uzbekistan U18
8,2602932,England U18,Canada U18,1,3,Balanced,33,804.0,24.63,41.96,45.71,-0.413012,-0.461168,2025-09-09,England U18-Canada U18
9,2602932,England U18,Canada U18,1,3,Control,31,844.0,25.86,16.94,18.76,0.526564,0.378465,2025-09-09,England U18-Canada U18


In [599]:
sql_test_phases_by_tier_metrics = 'SELECT * FROM `the-fa-performance-insights.dev_vanni.phases_by_tier_metrics`'
test_phases_by_tier_metrics = pandas_gbq.read_gbq(sql_test_phases_by_tier_metrics, project_id = 'the-fa-performance-insights')

display_all(test_phases_by_tier_metrics[(test_phases_by_tier_metrics['team_tier'] == 1) & (test_phases_by_tier_metrics['event_team_name'] == 'England U18')])

Downloading: 100%|█████████████████████████████████████████████████████████████|


,team_tier,against_tier,phase,time_diff,KPI,Value,n_of_matches,event_team_name,benchmark_type
144,1.0,1.0,BUILD,8388.299,% IN BUILD PHASE,14.05,57,England U18,England Benchmark
145,1.0,2.0,BUILD,10490.139,% IN BUILD PHASE,16.97,57,England U18,England Benchmark
146,1.0,3.0,BUILD,7185.271,% IN BUILD PHASE,15.17,57,England U18,England Benchmark
147,1.0,1.0,CREATE,14363.974,% IN CREATE PHASE,24.06,57,England U18,England Benchmark
148,1.0,2.0,CREATE,17114.765,% IN CREATE PHASE,27.69,57,England U18,England Benchmark
149,1.0,3.0,CREATE,13463.049,% IN CREATE PHASE,28.43,57,England U18,England Benchmark
150,1.0,1.0,FINISH,9178.738,% IN FINISH PHASE,15.37,57,England U18,England Benchmark
151,1.0,2.0,FINISH,11869.644,% IN FINISH PHASE,19.20,57,England U18,England Benchmark
152,1.0,3.0,FINISH,9227.301,% IN FINISH PHASE,19.48,57,England U18,England Benchmark
153,1.0,1.0,% IN HIGH BLOCK,9837.422,% IN HIGH BLOCK,16.48,57,England U18,England Benchmark
